In [ ]:
# he

In [5]:
import pandas as pd
# !pip install pandasql
from pandasql import sqldf

from datetime import datetime

from IPython.display import display

In [2]:
file_path='cx_dataset.xlsx'
titanic = pd.read_excel(file_path, sheet_name='d22_titanic')

In [3]:
sqldf("select * from titanic limit 2")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [10]:
query = '''
SELECT * 
FROM titanic 
LIMIT 2
'''

sqldf(query)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


#### SQL info

In [ ]:
# window function and group by are not used in the same query, can be used with cte. like define a cte and perfrom group by operation and the use that cte table to perfom window function operation.

#### [Q3-Premium Customers](https://100daysofsql.com/new-courses/1/coding-test/4)

In [ ]:
-- Namastekart, an e-commerce company want to start special reward program for their premium customers.  The customers who have placed more number of orders than the average number of orders placed by each customers are considered as premium customers.

-- Write an SQL to find the list of premium customers along with the number of orders placed by each of them. 

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_name	varchar(20)
-- sales	int
 

-- Example Input

-- orders

-- order_id	order_date	customer_name	sales
-- 1	2023-01-01	Alexa	1239
-- 2	2023-01-02	Alexa	1239
-- 3	2023-01-03	Alexa	1239
-- 4	2023-01-03	Alexa	1239
-- 5	2023-01-01	Ramesh	1239
-- 6	2023-01-02	Ramesh	1239
-- 7	2023-01-03	Ramesh	1239
-- 8	2023-01-03	Neha	1200
-- 9	2023-01-03	Subhash	100
-- 10	2023-01-03	Subhash	230
 

-- Expected Output:

-- customer_name	no_of_orders
-- Alexa	4
-- Ramesh	3
-- Explanation: There are total 10 orders placed by 4 customers so the average number order placed by each customers is 10/4 = 2.5 . Alexa and ramesh have done more than 2.5 orders so they are in the output. 

In [ ]:
-- #my_sol

with cte as 
(select count(*)/count(distinct(customer_name)) as avg_orders from orders)

-- select * from cte
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*)>cte.avg_orders

#provided_sol

select customer_name,count(*) as no_of_orders 
from orders
group by customer_name having count(*)>(select count(*)*1.0/count(distinct customer_name) as avg_no_of_orders  from orders)

**chat gpt**

In SQL, when you use a comma between two tables in the `FROM` clause, it is creating a Cartesian product (cross join) between those tables. This means that every row from the first table is paired with every row from the second table. In your query:

```sql
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*) > cte.avg_orders
```

The `orders` table and the common table expression (CTE) `cte` are combined using a cross join. Since `cte` is expected to return only one row (it contains an aggregate calculation), the result is effectively adding the `avg_orders` value to every row from the `orders` table. 

Here's a step-by-step breakdown of the query:

1. The CTE `cte` is defined, which calculates the average number of orders per customer:
   ```sql
   with cte as 
   (select count(*)/count(distinct(customer_name)) as avg_orders from orders)
   ```
   
2. The main query:
   ```sql
   select customer_name, count(*) as no_of_orders
   from orders, cte
   group by customer_name
   having count(*) > cte.avg_orders
   ```
   - `from orders, cte`: This creates a Cartesian product of `orders` and the single row from `cte`. Because `cte` returns a single row, each row of `orders` is effectively augmented with the `avg_orders` value.
   - `group by customer_name`: This groups the result by `customer_name`.
   - `having count(*) > cte.avg_orders`: This filters the groups to only include customers whose number of orders is greater than the average number of orders per customer.

So, while a join is not explicitly specified, the cross join ensures that each row of `orders` has access to the `avg_orders` value from the CTE. This allows the `having` clause to compare the count of orders for each customer against this average.

If the CTE were to return more than one row, this query would not work correctly because the Cartesian product would multiply the rows from `orders` by the number of rows in the CTE, leading to incorrect results. In this case, since the CTE is designed to return exactly one row, the query functions as intended.

#### [4 - LinkedIn Top Voice](https://100daysofsql.com/new-courses/1/coding-test/5)

In [ ]:
-- LinkedIn is a professional social networking app. They want to give top voice badge to their best creators to encourage them to create more quality content. A creator qualifies for the badge if he/she satisfies following criteria. 

-- 1- Creator should have more then 50k followers.

-- 2- Creator should have more than 100k impressions on the posts that they published in the month of Dec-2023.

-- 3- Creator should have published atleast 3 posts in Dec-2023.

-- Write a SQL to get the list of top voice creators name along with no of posts and impressions by them in the month of Dec-2023.

-- Table: creators(primary key : creator_id)

-- column name	data type 
-- creator_id 	int
-- creator_name 	varchar(20)
-- followers 	int

-- Table: posts(primary key : post_id)

-- column name	data type 
-- creator_id 	int
-- post_id 	varchar(3)
-- publish_date 	date
-- impressions 	int
 

-- Example output:

-- creator_name	no_of_posts	 total_impressions
-- Ankit Bansal	3	132000
 

In [ ]:
-- my_sol

select c.creator_name, count(*) as no_of_posts, sum(p.impressions) as total_impressions
from creators c join posts p on c.creator_id=p.creator_id
where c.followers>50000 and 
strftime('%Y%m', p.publish_date) = '202312'
group by c.creator_name
having sum(impressions)>100000 and 
count(*)>=3

-- site_sol
select c.creator_name,count(p.post_id) as no_of_posts, sum(p.impressions) as total_impressions
from creators c
inner join posts p on c.creator_id=p.creator_id
where c.followers>50000 and strftime('%Y%m', p.publish_date)='202312'
group by c.creator_name,c.followers
having sum(p.impressions)>100000 and count(p.post_id)>=3 

#### [5 - CIBIL Score](https://100daysofsql.com/new-courses/1/coding-test/6)

CIBIL score, often referred to as a credit score, is a numerical representation of an individual's creditworthiness.While the exact formula used by credit bureaus like CIBIL may not be publicly disclosed and can vary slightly between bureaus, the following are some common factors that typically influence the calculation of a credit score:

Payment History: This accounts for the largest portion of your credit score. It includes factors such as whether you pay your bills on time, any late payments, defaults, bankruptcies, etc. Assume this accounts for 70 percent of your credit score.

Credit Utilization Ratio: This is the ratio of your credit card balances to your credit limits. Keeping this ratio low (ideally below 30%) indicates responsible credit usage. Assume it accounts for 30% of your score and below logic to calculate it: 

           Utilization below 30% = 1

           Utilization between 30% and 50% = 0.7

           Utilization above 50% = 0.5

Assume that we have credit card bills data for March 2023 based on that we need to calculate credit utilization ratio.

 

Final Credit score formula = (on_time_loan_or_bill_payment)/total_bills_and_loans * 70 + Credit Utilization Ratio * 30 

 

Table: customers(primary key : customer_id )

column name	data type 
customer_id 	int
credit_limit 	int
 

Table: loans(primary key : loan_id )

column name	data type 
customer_id 	int
loan_id 	int
loan_due_date 	date
 

Table: credit_card_bills(primary key : bill_id )

column name	data type 
customer_id 	int
bill_id 	int
bill_due_date 	date
balance_amount 	int
 

Table: customer_transactions(primary key : loan_bill_id )

column name	data type 
loan_bill_id 	int
transaction_type 	varchar(10)
transaction_date	date
 

Expected Output: 

customer_id	cibil_score
1	82.5
2	91.0

#### [11 - New and Repeat Customers](https://100daysofsql.com/new-courses/1/coding-test/12)

In [ ]:
# NamasteKart an ecommerce company wants to build a very imporant business metrics where they want to track on daily basis how many new and repeat customers are purchasing products from their website. A new customer is defined when he purchased anything for the first time from the website and repeat customer is someone who has done atleast one purchase in the past.

# Table: customer_orders (primary key : order_id)

# column name	datatype
# order_id	int
# order_date	date
# customer_id 	int
# order_amount 	int

# Expected Output:

# order_date	new_customers	repeat_customers
# 2022-01-01	3	0
# 2022-01-02	2	1
# 2022-01-03	1	2
 

In [1]:
import pandas as pd

# Define the data as a list of dictionaries
data = [
    {"order_id": 1, "customer_id": 100, "order_date": "2022-01-01", "order_amount": 2000},
    {"order_id": 2, "customer_id": 200, "order_date": "2022-01-01", "order_amount": 2500},
    {"order_id": 3, "customer_id": 300, "order_date": "2022-01-01", "order_amount": 2100},
    {"order_id": 4, "customer_id": 100, "order_date": "2022-01-02", "order_amount": 2000},
    {"order_id": 5, "customer_id": 400, "order_date": "2022-01-02", "order_amount": 2200},
    {"order_id": 6, "customer_id": 500, "order_date": "2022-01-02", "order_amount": 2700},
    {"order_id": 7, "customer_id": 100, "order_date": "2022-01-03", "order_amount": 3000},
    {"order_id": 8, "customer_id": 400, "order_date": "2022-01-03", "order_amount": 1000},
    {"order_id": 9, "customer_id": 600, "order_date": "2022-01-03", "order_amount": 3000}
]

# Create a DataFrame
customer_orders = pd.DataFrame(data)

# Print the DataFrame
print(customer_orders)


   order_id  customer_id  order_date  order_amount
0         1          100  2022-01-01          2000
1         2          200  2022-01-01          2500
2         3          300  2022-01-01          2100
3         4          100  2022-01-02          2000
4         5          400  2022-01-02          2200
5         6          500  2022-01-02          2700
6         7          100  2022-01-03          3000
7         8          400  2022-01-03          1000
8         9          600  2022-01-03          3000


In [4]:
query = '''
with cte as
(select * ,
min(order_date) over(partition by customer_id ) as first_order
from customer_orders
)

select 
order_date,
sum(case when order_date=first_order then 1 else 0 end) as new_customers,
sum(case when order_date!=first_order then 1 else 0 end) as repeat_customers
from cte
group by order_date
'''

sqldf(query)

,order_date,new_customers,repeat_customers
0,2022-01-01,3,0
1,2022-01-02,2,1
2,2022-01-03,1,2


In [6]:
# -- # provided solution
query = '''
with first_order_date as 
(select customer_id,min(order_date) as first_order 
from customer_orders 
group by customer_id
)
select co.order_date 
,sum(case when co.order_date=fod.first_order then 1 else 0 end) as new_customers
,sum(case when co.order_date>fod.first_order then 1 else 0 end) as repeat_customers
from customer_orders co 
inner join first_order_date fod on co.customer_id=fod.customer_id
group by co.order_date 
'''

sqldf(query)

,order_date,new_customers,repeat_customers
0,2022-01-01,3,0
1,2022-01-02,2,1
2,2022-01-03,1,2


#### [12 - The Little Master](https://100daysofsql.com/new-courses/1/coding-test/13)

In [ ]:
-- Sachin Tendulkar - Also known as little master. You are given runs scored by Sachin in his first 10 matches. You need to write an SQL to get match number when he completed 500 runs and his batting average at the end of 10 matches.

-- Batting Average = (Total runs scored) / (no of times batsman got out)

-- Round the result to 2 decimal places.

-- Table: sachin(primary key : match_no)

-- column name	datatype
-- match_no	int
-- runs_scored	int
-- status	varchar(10)
 
-- Expected Output:

-- match_no	batting_average
-- 8	70.33

In [7]:
import pandas as pd

# Your SQL data
data = [
    (1, 53, 'out'),
    (2, 59, 'not-out'),
    (3, 113, 'out'),
    (4, 29, 'out'),
    (5, 0, 'out'),
    (6, 39, 'out'),
    (7, 73, 'out'),
    (8, 149, 'out'),
    (9, 93, 'out'),
    (10, 25, 'out')
]

# Create a DataFrame
columns = ['match_no', 'runs_scored', 'status']
sachin = pd.DataFrame(data, columns=columns)

# Print the DataFrame
print(sachin)


   match_no  runs_scored   status
0         1           53      out
1         2           59  not-out
2         3          113      out
3         4           29      out
4         5            0      out
5         6           39      out
6         7           73      out
7         8          149      out
8         9           93      out
9        10           25      out


In [ ]:
# -- my sol

query = '''
with batting_average as
(
    select round(sum(runs_scored)*1.0/sum(case when status='out' then 1 else 0 end),2) as batting_average from sachin

)
, running_score as
(
    select *, 
    sum(runs_scored) over(rows between unbounded preceding and current row) as running_score
    from sachin, batting_average
)
select match_no, batting_average
from running_score
where running_score>=500
limit 1
'''

sqldf(query)

In [ ]:
# provided sol

query = '''
with cte as (
select *
,sum(runs_scored) over()*1.0 / sum(case when status='out' then 1 else 0 end) over() as batting_average
,sum(runs_scored) over(order by match_no rows between unbounded preceding and current row) as running_sum
from sachin
)
select match_no,round(batting_average,2) as batting_average
from (select *, row_number() over(order by running_sum) as rn
from cte where running_sum>500)
where rn=1;
'''

sqldf(query)

#### [13 - Trending Products](https://100daysofsql.com/new-courses/1/coding-test/14)

In [ ]:
# Amazon wants to find out the Trending products for each month. Trending products are those for which any given month sales is more than the sum of previous 2 months sales for that product.
# Please note that for first 2 months of operations this metrics does not make sense. So output should start from 3rd month only.  Assume that each product has atleast 1 sales each month.

# Table: orders (primary key : order_month, product_id)

# column name	datatype
# order_month	date
# product_id	varchar(5)
# sales	int
 

# Expected Output:

# order_month	product_id
# 202303	p1
# 202304	p1
# 202304	p2

In [3]:
import pandas as pd

# Define the data
data = {
    'order_month': ['2023-01-01', '2023-01-01', '2023-02-01', '2023-02-01', '2023-03-01', '2023-03-01', '2023-04-01', '2023-04-01'],
    'product_id': ['p1', 'p2', 'p1', 'p2', 'p1', 'p2', 'p1', 'p2'],
    'sales': [100, 500, 700, 300, 900, 700, 2000, 1100]
}

# Create the DataFrame
orders = pd.DataFrame(data)

# Display the DataFrame
print(orders)


  order_month product_id  sales
0  2023-01-01         p1    100
1  2023-01-01         p2    500
2  2023-02-01         p1    700
3  2023-02-01         p2    300
4  2023-03-01         p1    900
5  2023-03-01         p2    700
6  2023-04-01         p1   2000
7  2023-04-01         p2   1100


In [4]:
query = '''
with cte as 
(select *,
lag(sales,1) over(partition by product_id order by order_month)  as prev1,
lag(sales,2) over(partition by product_id order by order_month) as prev2,
lag(sales,1) over(partition by product_id order by order_month)+lag(sales,2) over(partition by product_id order by order_month) as prev_total
from orders)

select order_month,product_id
from cte
where prev_total is not null and prev_total<sales;
'''
sqldf(query)

,order_month,product_id
0,2023-03-01,p1
1,2023-04-01,p1
2,2023-04-01,p2


In [ ]:
-- code to insert the orders table

-- Step 1: Create the table
CREATE TABLE orders (
    order_month DATE,
    product_id VARCHAR(5),
    sales INT,
    PRIMARY KEY (order_month, product_id)
);


-- Step 2: Insert the data
INSERT INTO orders (order_month, product_id, sales) VALUES
('2023-01-01', 'p1', 100),
('2023-01-01', 'p2', 500),
('2023-02-01', 'p1', 700),
('2023-02-01', 'p2', 300),
('2023-03-01', 'p1', 900),
('2023-03-01', 'p2', 700),
('2023-04-01', 'p1', 2000),
('2023-04-01', 'p2', 1100);


In [ ]:
with cte as 
(select *,
lag(sales,1) over(partition by product_id order by order_month)  as prev1,
lag(sales,2) over(partition by product_id order by order_month) as prev2,
lag(sales,1) over(partition by product_id order by order_month)+lag(sales,2) over(partition by product_id order by order_month) as prev_total
from orders)

select order_month,product_id
from cte
where prev_total is not null and prev_total<sales;

In [ ]:
with cte2 as
(select *,
sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as prev_total

from orders)

select 
order_month, product_id

from cte2
where sales>prev_total and order_month not in (select distinct order_month from orders order by order_month asc limit 2)


In [ ]:
-- provided solution
with cte as (
select * 
,sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as last2
,row_number() over(partition by product_id order by order_month) as rn
from orders 
)
select order_month,product_id from cte 
where rn>=3 and sales > last2

#### [14 - Uber Rides](https://100daysofsql.com/new-courses/1/coding-test/15)

In [ ]:
# A profit ride for a Uber driver is considered when the start location and start time of a ride exactly match with the previous ride's end location and end time. 

# Write an SQL to calculate total number of rides and total profit rides by each driver.

# Table: drivers(primary key : id , start_time )

# column name	datatype
# id 	varchar(10)
# start_time 	time
# end_time 	time
# start_loc 	varchar(1)
# end_loc 	varchar(1)
 

# Expected output :

# id	total_rides	profit_rides
# dri_1	5	1
# dri_2	2	0

In [5]:
import pandas as pd

# Define the data
data = {
    'id': ['dri_1', 'dri_1', 'dri_1', 'dri_1', 'dri_1', 'dri_2', 'dri_2'],
    'start_time': ['09:00:00', '09:30:00', '11:00:00', '12:00:00', '13:30:00', '12:15:00', '13:30:00'],
    'end_time': ['09:30:00', '10:30:00', '11:30:00', '12:30:00', '14:30:00', '12:30:00', '14:30:00'],
    'start_loc': ['a', 'b', 'd', 'f', 'c', 'f', 'c'],
    'end_loc': ['b', 'c', 'e', 'g', 'h', 'g', 'h']
}

# Create the DataFrame
drivers = pd.DataFrame(data)

# Display the DataFrame
print(drivers)


      id start_time  end_time start_loc end_loc
0  dri_1   09:00:00  09:30:00         a       b
1  dri_1   09:30:00  10:30:00         b       c
2  dri_1   11:00:00  11:30:00         d       e
3  dri_1   12:00:00  12:30:00         f       g
4  dri_1   13:30:00  14:30:00         c       h
5  dri_2   12:15:00  12:30:00         f       g
6  dri_2   13:30:00  14:30:00         c       h


In [12]:
query = '''
with cte as
(select *
,lead(start_time,1) over(partition by id order by start_time) as next_time
,lead(start_loc ,1) over(partition by id order by start_loc ) as next_loc
from drivers)

select 
id
, count(*) as total_rides

, sum(case when end_time=next_time and end_loc=next_loc then 1 else 0 end) as profit_rides

from cte
group by id
'''
sqldf(query)

,id,total_rides,profit_rides
0,dri_1,5,1
1,dri_2,2,0


In [ ]:
code to create the table 

-- Create the drivers table
CREATE TABLE drivers (
    id VARCHAR(10),
    start_time TIME,
    end_time TIME,
    start_loc VARCHAR(1),
    end_loc VARCHAR(1),
    PRIMARY KEY (id, start_time)
);

-- Insert data into the drivers table
INSERT INTO drivers (id, start_time, end_time, start_loc, end_loc) VALUES
('dri_1', '09:00:00', '09:30:00', 'a', 'b'),
('dri_1', '09:30:00', '10:30:00', 'b', 'c'),
('dri_1', '11:00:00', '11:30:00', 'd', 'e'),
('dri_1', '12:00:00', '12:30:00', 'f', 'g'),
('dri_1', '13:30:00', '14:30:00', 'c', 'h'),
('dri_2', '12:15:00', '12:30:00', 'f', 'g'),
('dri_2', '13:30:00', '14:30:00', 'c', 'h');


In [ ]:
with cte as
(select *,
lead(start_time,1) over(partition by id order by start_time) as nxt_ride,
lead(start_loc,1) over(partition by id order by start_time) as nxt_start

from drivers)

select id, count(*) as total_rides,
sum(case when end_time=nxt_ride and end_loc=nxt_start then 1 else 0 end) as profit_rides
from cte
group by id


In [ ]:
-- provided solution

with cte as (
select * 
, lag(end_time,1) over(partition by id order by start_time) as prev_end_time
, lag(end_loc,1) over(partition by id order by start_time) as prev_end_loc
from drivers
)
select id, count(*) as total_rides , 
sum(case when start_time = prev_end_time and start_loc=prev_end_loc then 1 else 0 end) as profit_rides
from cte 
group by id

#### [15 - The United States of America](https://100daysofsql.com/new-courses/1/coding-test/16)

In [ ]:
''' In some poorly designed UI applications, there's often a lack of data input restrictions. For instance, in a free text field for the country, users might input variations such as 'USA,' 'United States of America,' or 'US.'

Suppose we have survey data from individuals in the USA about their job satisfaction, rated on a scale of 1 to 5. Write a SQL query to count the number of respondents for each rating on the scale. Additionally, include the country name in the format that occurs most frequently in that scale.

Table: survey (primary key : name)

column name	datatype
name 	varchar(10)
job_satisfaction 	int
country 	varchar(20)

Expected output:

job_satisfaction | country | number_of_respondents
4	USA	3
5	United States	4

'''

In [15]:
import pandas as pd

# Create the DataFrame
data = {
    'name': ['Alex', 'Saurabh', 'Mark', 'Shane', 'Kim', 'Joe', 'Mira'],
    'job_satisfaction': [4, 5, 4, 4, 5, 5, 5],
    'country': ['USA', 'US', 'United States', 'USA', 'United States', 'USA', 'United States']
}

survey = pd.DataFrame(data)

# Display the DataFrame
display(survey)


,name,job_satisfaction,country
0,Alex,4,USA
1,Saurabh,5,US
2,Mark,4,United States
3,Shane,4,USA
4,Kim,5,United States
5,Joe,5,USA
6,Mira,5,United States


In [22]:
query = '''
with cte as 
(select 
job_satisfaction, country
,count(*)  as ct
from survey
group by job_satisfaction, country)

, cte2 as
(select *, 
dense_rank() over(partition by job_satisfaction order by ct desc) as dr
, sum(ct) over(partition by job_satisfaction) as no_of_respondents
from cte)

select job_satisfaction , country, no_of_respondents
from cte2
where dr=1
'''
sqldf(query)

,job_satisfaction,country,no_of_respondents
0,4,USA,3
1,5,United States,4


In [ ]:
-- -- Create the survey table
-- CREATE TABLE survey (
--     name VARCHAR(10) PRIMARY KEY,
--     job_satisfaction INT,
--     country VARCHAR(20)
-- );

-- -- Insert data into the survey table
-- INSERT INTO survey (name, job_satisfaction, country) VALUES
-- ('Alex', 4, 'USA'),
-- ('Saurabh', 5, 'US'),
-- ('Mark', 4, 'United States'),
-- ('Shane', 4, 'USA'),
-- ('Kim', 5, 'United States'),
-- ('Joe', 5, 'USA'),
-- ('Mira', 5, 'United States');


In [ ]:
with cte as (
select country,job_satisfaction,count(*) as cnt 
from survey
group by country,job_satisfaction
)
, cte2 as (
select *, sum(cnt) over(partition by job_satisfaction) as number_of_respondents
,max(cnt) over(partition by job_satisfaction) as max_cnt
from cte
)
select job_satisfaction,country,number_of_respondents from cte2 
where cnt=max_cnt


In [ ]:
-- my_sol
-- select  * from survey order by job_satisfaction;

with responses as
(
    select job_satisfaction, count(*) as number_of_respondents
    from survey
    group by job_satisfaction
)
, countries as
(
    select job_satisfaction, country, count(*) as country_count
    from survey
    group by job_satisfaction, country

)
, countries_rank as
(
    select job_satisfaction, country, country_count,
    dense_rank() over(partition by job_satisfaction order by country_count desc) as dr
    from countries
)
-- select * from countries_rank

select r.job_satisfaction, cr.country, number_of_respondents
from responses r join countries_rank cr on r.job_satisfaction=cr.job_satisfaction
where dr=1


#### [16 - Product Recommendation](https://100daysofsql.com/new-courses/1/coding-test/17)

In [ ]:
'''
Product recommendation. Just the basic type (“customers who bought this also bought…”). That, in its simplest form, is an outcome of basket analysis. Write a SQL to find the product pairs along with the count of times they have been purchased together. Based on the history ecommerce website can recommend products to new user.

Table: orders (primary key : order_id)

column name	datatype
order_id	int
customer_id 	int
product_id 	varchar(2)

Expected Output:

product_1	product_2	purchase_frequency
p2	p1	2
p3	p1	1
p3	p2	1
p4	p1	1
p4	p2	1
Note : In the output product_1 values should be greater than product_2 values. eg: pair p1,p2 or p2,p1 represent same thing but we need to show p2,p1 in the output.

'''

In [24]:
import pandas as pd

# Define the data as a list of dictionaries
data = [
    {'order_id': 1, 'customer_id': 1, 'product_id': 'p1'},
    {'order_id': 1, 'customer_id': 1, 'product_id': 'p2'},
    {'order_id': 1, 'customer_id': 1, 'product_id': 'p3'},
    {'order_id': 2, 'customer_id': 2, 'product_id': 'p1'},
    {'order_id': 2, 'customer_id': 2, 'product_id': 'p2'},
    {'order_id': 2, 'customer_id': 2, 'product_id': 'p4'},
    {'order_id': 3, 'customer_id': 1, 'product_id': 'p5'}
]

# Create the DataFrame
orders = pd.DataFrame(data)

# Display the DataFrame
print(orders)



   order_id  customer_id product_id
0         1            1         p1
1         1            1         p2
2         1            1         p3
3         2            2         p1
4         2            2         p2
5         2            2         p4
6         3            1         p5


In [25]:
query = '''
select 
o1.product_id, o2.product_id, count(*)
from orders o1 cross join orders o2 on
o1.order_id=o2.order_id
where o1.product_id>o2.product_id
group by o1.product_id, o2.product_id
'''
sqldf(query)

,product_id,product_id,count(*)
0,p2,p1,2
1,p3,p1,1
2,p3,p2,1
3,p4,p1,1
4,p4,p2,1


In [ ]:
-- CREATE TABLE orders (
--     order_id INT NOT NULL,
--     customer_id INT NOT NULL,
--     product_id VARCHAR(2) NOT NULL,
--     PRIMARY KEY (order_id, product_id)
-- );

-- INSERT INTO orders (order_id, customer_id, product_id) VALUES
-- (1, 1, 'p1'),
-- (1, 1, 'p2'),
-- (1, 1, 'p3'),
-- (2, 2, 'p1'),
-- (2, 2, 'p2'),
-- (2, 2, 'p4'),
-- (3, 1, 'p5');

In [ ]:
-- -- my sol
-- select 
-- o1.product_id, o2.product_id, count(*) as purchase_frequency
-- from orders o1 cross join orders o2
-- on o1.order_id=o2.order_id and o1.product_id!=o2.product_id and o1.product_id>o2.product_id
-- group by o1.product_id, o2.product_id


In [ ]:
-- select 
--     o1.product_id as product_1, o2.product_id as product_2,
--     count(*) as purchase_frequency
-- from 
--     orders o1 
--     join orders o2 on o1.order_id=o2.order_id
-- where 
--     o1.product_id>o2.product_id
-- group by 
--     o1.product_id, o2.product_id

#### [17 - Account Balance](https://100daysofsql.com/new-courses/1/coding-test/18)

In [ ]:
-- You are given a list of users and their opening account balance along with the trasaction done by them. Write a SQL to calculate their account balance at the end of all the transactions. Please note that users can do transactions among themselves as well. 

-- Table: users(primary key : user_id)

-- column name	datatype
-- user_id 	int
-- username 	varchar(10)
-- opening_balance 	int
 

-- Table: transactions (primary key : id)

-- column name	datatype
-- id	int
-- from_userid 	int
-- to_userid  	int
-- amount 	int

-- Expected Output:

-- user_id	final_balance
-- Ankit	2000
-- Rahul	10200
-- Amit	2800
-- Agam	7500

In [28]:
import pandas as pd

# Data for the users table
users_data = [
    {'user_id': 100, 'username': 'Ankit', 'opening_balance': 1000},
    {'user_id': 101, 'username': 'Rahul', 'opening_balance': 9000},
    {'user_id': 102, 'username': 'Amit', 'opening_balance': 5000},
    {'user_id': 103, 'username': 'Agam', 'opening_balance': 7500}
]

# Data for the transactions table
transactions_data = [
    {'id': 1, 'from_userid': 100, 'to_userid': 102, 'amount': 500},
    {'id': 2, 'from_userid': 102, 'to_userid': 101, 'amount': 700},
    {'id': 3, 'from_userid': 101, 'to_userid': 102, 'amount': 600},
    {'id': 4, 'from_userid': 102, 'to_userid': 100, 'amount': 1500},
    {'id': 5, 'from_userid': 102, 'to_userid': 101, 'amount': 800},
    {'id': 6, 'from_userid': 102, 'to_userid': 101, 'amount': 300}
]

# Create the DataFrames
users = pd.DataFrame(users_data)
transactions = pd.DataFrame(transactions_data)

# show the DataFrames
print("Users DataFrame:")
display(users)
print("\nTransactions DataFrame:")
display(transactions)


Users DataFrame:


,user_id,username,opening_balance
0,100,Ankit,1000
1,101,Rahul,9000
2,102,Amit,5000
3,103,Agam,7500



Transactions DataFrame:


,id,from_userid,to_userid,amount
0,1,100,102,500
1,2,102,101,700
2,3,101,102,600
3,4,102,100,1500
4,5,102,101,800
5,6,102,101,300


In [29]:
query = '''
with all_transactions as 
(select from_userid as user_id, amount*-1 as amount from transactions
union all
select to_userid as user_id, amount as amount from transactions)

-- select * from cte

, userwise_transactions as 
(select user_id, sum(amount) as amount from all_transactions
group by user_id)

select 
u.username, opening_balance+coalesce(amount,0) as final_balance

from users u left join userwise_transactions on u.user_id=userwise_transactions.user_id 
'''
sqldf(query)

,user_id,final_balance
0,100,2000
1,101,10200
2,102,2800
3,103,7500


In [30]:
# my solution 
query = '''

with amt_sent as
(
    select from_userid, sum(amount) as sent
    from transactions 
    group by from_userid
)
, amt_received as
(
    select to_userid, sum(amount) as received
    from transactions 
    group by to_userid
)

select 
username as user_id, opening_balance-coalesce(sent,0)+coalesce(received,0) as final_balance

from users u left join amt_sent on u.user_id=amt_sent.from_userid 
left join amt_received on u.user_id=amt_received.to_userid
'''
sqldf(query)

,user_id,final_balance
0,Ankit,2000
1,Rahul,10200
2,Amit,2800
3,Agam,7500


In [ ]:
-- -- Creating the users table
-- CREATE TABLE users (
--     user_id INTEGER PRIMARY KEY,
--     username VARCHAR(10) NOT NULL,
--     opening_balance INTEGER NOT NULL
-- );

-- -- Creating the transactions table
-- CREATE TABLE transactions (
--     id INTEGER PRIMARY KEY,
--     from_userid INTEGER NOT NULL,
--     to_userid INTEGER NOT NULL,
--     amount INTEGER NOT NULL,
--     FOREIGN KEY (from_userid) REFERENCES users(user_id),
--     FOREIGN KEY (to_userid) REFERENCES users(user_id)
-- );

-- -- Inserting data into users table
-- INSERT INTO users (user_id, username, opening_balance) VALUES
-- (100, 'Ankit', 1000),
-- (101, 'Rahul', 9000),
-- (102, 'Amit', 5000),
-- (103, 'Agam', 7500);

-- -- Inserting data into transactions table
-- INSERT INTO transactions (id, from_userid, to_userid, amount) VALUES
-- (1, 100, 102, 500),
-- (2, 102, 101, 700),
-- (3, 101, 102, 600),
-- (4, 102, 100, 1500),
-- (5, 102, 101, 800),
-- (6, 102, 101, 300);


In [ ]:
-- my solution

with amt_sent as
(
    select from_userid, sum(amount) as sent
    from transactions 
    group by from_userid
)
, amt_received as
(
    select to_userid, sum(amount) as received
    from transactions 
    group by to_userid
)


select 
username as user_id, opening_balance-coalesce(sent,0)+coalesce(received,0) as final_balance

from users u left join amt_sent on u.user_id=amt_sent.from_userid 
left join amt_received on u.user_id=amt_received.to_userid

#### [18 - boAt Lifestyle](https://100daysofsql.com/new-courses/1/coding-test/19)

In [ ]:
-- boAt Lifestyle is focusing on influencer marketing to build and scale their brand. They want to partner with power creators for their upcoming campaigns. The creators should satisy below conditions to qualify :

-- 1- They should have 100k+ followers on atleast 2 social media platforms and

-- 2- They should have atleast 50k+ views on their latest YouTube video.

-- Write an SQL to get creator id and name satisfying above conditions.

-- Table: creators(primary key : order_id)

-- column name	datatype
-- id 	int
-- name 	varchar(10)
-- platform 	varchar(10)
-- followers 	int

-- Table: youtube_videos(primary key : order_id)

-- column name	datatype
-- id 	int
-- creator_id 	int
-- publish_date 	date
-- views	int
 

-- Expected Output:

-- id	name
-- 102	Dhruv

In [31]:
import pandas as pd

# Data for creators table
creators_data = {
    'id': [100, 100, 101, 101, 101, 102, 102, 102],
    'name': ['Ankit', 'Ankit', 'Warikoo', 'Warikoo', 'Warikoo', 'Dhruv', 'Dhruv', 'Dhruv'],
    'platform': ['YouTube', 'LinkedIn', 'YouTube', 'LinkedIn', 'Instagram', 'LinkedIn', 'YouTube', 'Instagram'],
    'followers': [90000, 150000, 500000, 600000, 800000, 60000, 900000, 800000]
}

# Data for youtube_videos table
youtube_videos_data = {
    'id': [1, 2, 3, 4, 5, 6],
    'creator_id': [100, 100, 101, 101, 102, 102],
    'publish_date': ['2024-01-01', '2024-01-06', '2024-01-05', '2024-01-07', '2024-01-05', '2024-01-09'],
    'views': [52000, 62000, 59000, 22000, 70000, 90000]
}

# Creating DataFrames
creators = pd.DataFrame(creators_data)
youtube_videos = pd.DataFrame(youtube_videos_data)

# show the DataFrames
display("Creators DataFrame:")
print(creators)
print("\nYouTube Videos DataFrame:")
display(youtube_videos)


'Creators DataFrame:'

    id     name   platform  followers
0  100    Ankit    YouTube      90000
1  100    Ankit   LinkedIn     150000
2  101  Warikoo    YouTube     500000
3  101  Warikoo   LinkedIn     600000
4  101  Warikoo  Instagram     800000
5  102    Dhruv   LinkedIn      60000
6  102    Dhruv    YouTube     900000
7  102    Dhruv  Instagram     800000

YouTube Videos DataFrame:


,id,creator_id,publish_date,views
0,1,100,2024-01-01,52000
1,2,100,2024-01-06,62000
2,3,101,2024-01-05,59000
3,4,101,2024-01-07,22000
4,5,102,2024-01-05,70000
5,6,102,2024-01-09,90000


In [32]:
# -- sol 1
query = '''
with followers_tbl as 
(select * ,
sum(case when followers>=100000 then 1 else 0 end) over(partition by id) as follower_flag
from creators)

,videos_tbl as 
(select *,
max(publish_date) over(partition by creator_id) as latest_video_date
from youtube_videos)

select 
distinct f.id, name
from followers_tbl as f join videos_tbl as v on f.id=v.creator_id
where views>=50000 and follower_flag>=2 and publish_date=latest_video_date
'''
sqldf(query)

,id,name
0,102,Dhruv


In [33]:
# sol 2
query = '''
with condition_1 as (
select id,name
 from creators
 where followers > 100000
 group by id,name
 having count(*)>=2
)
,condition_2 as 
(
select *, row_number() over(partition by creator_id order by publish_date desc) as rn    
from youtube_videos
)
select c1.id,c1.name from condition_2 c2
inner join condition_1 c1 on c1.id=c2.creator_id
where rn=1 and views>50000
'''
sqldf(query)

,id,name
0,102,Dhruv


In [ ]:
-- -- Creating the creators table
-- CREATE TABLE creators (
--     id INTEGER NOT NULL,
--     name VARCHAR(10) NOT NULL,
--     platform VARCHAR(10) NOT NULL,
--     followers INTEGER NOT NULL,
--     PRIMARY KEY (id, platform)
-- );

-- -- Creating the youtube_videos table
-- CREATE TABLE youtube_videos (
--     id INTEGER PRIMARY KEY,
--     creator_id INTEGER NOT NULL,
--     publish_date DATE NOT NULL,
--     views INTEGER NOT NULL,
--     FOREIGN KEY (creator_id) REFERENCES creators(id)
-- );

-- -- Inserting data into creators table
-- INSERT INTO creators (id, name, platform, followers) VALUES
-- (100, 'Ankit', 'YouTube', 90000),
-- (100, 'Ankit', 'LinkedIn', 150000),
-- (101, 'Warikoo', 'YouTube', 500000),
-- (101, 'Warikoo', 'LinkedIn', 600000),
-- (101, 'Warikoo', 'Instagram', 800000),
-- (102, 'Dhruv', 'LinkedIn', 60000),
-- (102, 'Dhruv', 'YouTube', 900000),
-- (102, 'Dhruv', 'Instagram', 800000);

-- -- Inserting data into youtube_videos table
-- INSERT INTO youtube_videos (id, creator_id, publish_date, views) VALUES
-- (1, 100, '2024-01-01', 52000),
-- (2, 100, '2024-01-06', 62000),
-- (3, 101, '2024-01-05', 59000),
-- (4, 101, '2024-01-07', 22000),
-- (5, 102, '2024-01-05', 70000),
-- (6, 102, '2024-01-09', 90000);


#### [19 - Dynamic Pricing](https://100daysofsql.com/new-courses/1/coding-test/20)

In [ ]:
-- You are given a products table where a new row is inserted every time the price of a product changes. Additionally, there is a transaction table containing details such as order_date and product_ID for each order. Write an SQL query to calculate the total sales value for each product, considering the cost of the product at the time of the order date.

-- Table: products(primary key : product_id)

-- column name	datatype
-- product_id	int
-- price_date	date
-- price	int

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- product_id	int

-- Expected output:

-- product_id	total_sales
-- 100	510
-- 101	3450



In [34]:
import pandas as pd

# Creating the products DataFrame
products_data = {
    'product_id': [100, 100, 100, 101, 101, 101],
    'price_date': ['2024-01-01', '2024-01-21', '2024-02-01', '2024-01-01', '2024-01-27', '2024-02-05'],
    'price': [150, 170, 190, 1000, 1200, 1250]
}
products = pd.DataFrame(products_data)
products['price_date'] = pd.to_datetime(products['price_date'])

# Creating the orders DataFrame
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': ['2024-01-05', '2024-01-21', '2024-02-20', '2024-01-07', '2024-02-04', '2024-02-05'],
    'product_id': [100, 100, 100, 101, 101, 101]
}
orders = pd.DataFrame(orders_data)
orders['order_date'] = pd.to_datetime(orders['order_date'])

print("Products DataFrame:")
display(products)
print("\nOrders DataFrame:")
display(orders)


Products DataFrame:


,product_id,price_date,price
0,100,2024-01-01,150
1,100,2024-01-21,170
2,100,2024-02-01,190
3,101,2024-01-01,1000
4,101,2024-01-27,1200
5,101,2024-02-05,1250



Orders DataFrame:


,order_id,order_date,product_id
0,1,2024-01-05,100
1,2,2024-01-21,100
2,3,2024-02-20,100
3,4,2024-01-07,101
4,5,2024-02-04,101
5,6,2024-02-05,101


In [35]:
# method-1
query = '''
WITH cte AS (
    SELECT *
         , LEAD(price_date, 1) OVER (PARTITION BY product_id ORDER BY price_date) AS valid_date
    FROM products
)
-- select * from cte
SELECT 
    cte.product_id,
    SUM(CASE 
            WHEN o.order_date >= cte.price_date AND (o.order_date < cte.valid_date OR cte.valid_date IS NULL) 
            THEN cte.price 
            ELSE 0 
        END) AS total_sales
FROM cte 
JOIN orders o ON cte.product_id = o.product_id 
GROUP BY cte.product_id;

'''
sqldf(query)

,product_id,total_sales
0,100,510
1,101,3450


In [36]:
# -- method-2
query = '''
WITH cte AS (
    SELECT *,
           LEAD(price_date, 1) OVER (PARTITION BY product_id ORDER BY price_date) AS valid_date
    FROM products
)

SELECT 
    o.product_id,
    SUM(p.price) AS total_sales
FROM orders o
JOIN cte p ON o.product_id = p.product_id
WHERE o.order_date >= p.price_date AND (o.order_date < p.valid_date OR p.valid_date IS NULL)
GROUP BY o.product_id;

'''
sqldf(query)

,product_id,total_sales
0,100,510
1,101,3450


In [37]:
query = '''
with cte as
(
    select 
        *, julianday(order_date)-julianday(price_date) as days_diff
    from 
        orders o 
        left join products p on o.order_date>=p.price_date and o.product_id=p.product_id
)
, cte2 as 
(
    select 
        *, dense_rank() over(partition by product_id, order_date order by days_diff asc) as dr
    from  cte
)
select product_id, sum(price) as total_sales
from cte2
where dr=1
group by product_id
'''
sqldf(query)

,product_id,total_sales
0,100,510
1,101,3450


In [38]:
# -- provided soln
query = '''
with price_range as (
select *
, date(lead(price_date,1,'9999-12-31') over(partition by product_id order by price_date),'-1 days') as price_end_date
 from products
)
select p.product_id , sum(p.price) as total_sales
from orders o 
inner join price_range p on o.product_id=p.product_id 
and o.order_date between p.price_date and p.price_end_date
group by p.product_id;
'''
sqldf(query)

,product_id,total_sales
0,100,510
1,101,2250


In [ ]:

-- -- Creating the products table
-- CREATE TABLE products (
--     product_id INTEGER NOT NULL,
--     price_date DATE NOT NULL,
--     price INTEGER NOT NULL,
--     PRIMARY KEY (product_id, price_date)
-- );

-- -- Creating the orders table
-- CREATE TABLE orders (
--     order_id INTEGER PRIMARY KEY,
--     order_date DATE NOT NULL,
--     product_id INTEGER NOT NULL,
--     FOREIGN KEY (product_id) REFERENCES products(product_id)
-- );
-- -- Inserting data into products table
-- INSERT INTO products (product_id, price_date, price) VALUES
-- (100, '2024-01-01', 150),
-- (100, '2024-01-21', 170),
-- (100, '2024-02-01', 190),
-- (101, '2024-01-01', 1000),
-- (101, '2024-01-27', 1200),
-- (101, '2024-02-05', 1250);

-- -- Inserting data into orders table
-- INSERT INTO orders (order_id, order_date, product_id) VALUES
-- (1, '2024-01-05', 100),
-- (2, '2024-01-21', 100),
-- (3, '2024-02-20', 100),
-- (4, '2024-01-07', 101),
-- (5, '2024-02-04', 101),
-- (6, '2024-02-05', 101);


#### [20 - Income Tax Returns](https://100daysofsql.com/new-courses/1/coding-test/21)

In [ ]:
-- Income tax returns are supposed to file every year before due date announce by govt every year . Goverment wants to find out people who have filed late returns or people who have completely skipped filing returns. 

-- Write an SQL to list all the users along with the year for which they have done late filing or missed filing the returns. Create a comment column as well specifying if its a 'late return' or  'missed'.

-- Table: income_tax_dates(primary key : financial_year)

-- column name	datatype
-- financial_year 	varchar(4)
-- file_start_date 	date
-- file_due_date 	date
 

-- Table: users (primary key : user_id,financial_year)

-- column name	datatype
-- user_id 	int
-- financial_year 	varchar(4)
-- return_file_date 	date
 


-- Expected Output:

-- user_id	financial_year	comment
-- 1	FY21	late return
-- 1	FY22	missed
-- 2	FY23	late return
 

In [26]:
# Creating the income_tax_dates DataFrame
income_tax_dates_data = {
    'financial_year': ['FY20', 'FY21', 'FY22', 'FY23'],
    'file_start_date': pd.to_datetime(['2020-05-01', '2021-06-01', '2022-05-05', '2023-05-05']),
    'file_due_date': pd.to_datetime(['2020-08-31', '2021-09-30', '2022-08-29', '2023-08-31'])
}
income_tax_dates = pd.DataFrame(income_tax_dates_data)

# Creating the users DataFrame
users_data = {
    'user_id': [1, 1, 1, 2, 2, 2, 2],
    'financial_year': ['FY20', 'FY21', 'FY23', 'FY20', 'FY21', 'FY22', 'FY23'],
    'return_file_date': pd.to_datetime(['2020-05-10', '2021-10-10', '2023-08-20', '2020-05-15', '2021-09-10', '2022-08-20', '2023-10-10'])
}
users = pd.DataFrame(users_data)

display(income_tax_dates)
display(users)


,financial_year,file_start_date,file_due_date
0,FY20,2020-05-01,2020-08-31
1,FY21,2021-06-01,2021-09-30
2,FY22,2022-05-05,2022-08-29
3,FY23,2023-05-05,2023-08-31


,user_id,financial_year,return_file_date
0,1,FY20,2020-05-10
1,1,FY21,2021-10-10
2,1,FY23,2023-08-20
3,2,FY20,2020-05-15
4,2,FY21,2021-09-10
5,2,FY22,2022-08-20
6,2,FY23,2023-10-10


In [28]:
query = '''
with all_users_years as 
(
select 
u.user_id,it.financial_year,it.file_due_date

from users u cross join income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)

select * from 
(select  -- *
a.user_id, a.financial_year, 
case when u.return_file_date>a.file_due_date then 'late return'
when u.return_file_date<a.file_due_date then 'on_time'
else 'missed' end as comment
from all_users_years a left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year) a
where comment !='on_time'
'''

sqldf(query)

,user_id,financial_year,comment
0,1,FY21,late return
1,1,FY22,missed
2,2,FY23,late return


In [ ]:
-- my soln
with all_users_years as 
(
select 
u.user_id,it.financial_year,it.file_due_date

from users u , income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)

select * from 
(select  -- *
a.user_id, a.financial_year, 
case when u.return_file_date>a.file_due_date then 'late return'
when u.return_file_date<a.file_due_date then 'on_time'
else 'missed' end as comment
from all_users_years a left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year) a
where comment !='on_time'

In [ ]:
-- provided solution
with all_users_years as 
(
select u.user_id,it.financial_year,it.file_due_date
from users u cross join income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)
select a.user_id,a.financial_year
,case when u.return_file_date > a.file_due_date then 'late return' else 'missed' end as comment
from all_users_years a 
left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year 
where u.return_file_date > a.file_due_date or u.return_file_date is null;

#### [21 - Malware Detection](https://100daysofsql.com/new-courses/1/coding-test/22)

In [ ]:
-- There are multiple antivirus softwares which are running on the systems and you have the data of how many malware they have detected  in each run.  You need to find out how many malware each software has detected in their latest run and what is the difference between the number of malwares detected in latest run and the second last run for each software. 

-- Please note that list only the softwares which have run for atleast 2 times and have detected atleast 10 malware in the latest run.

-- Table: malware(primary key : software_id,run_date)

-- column name	datatype
-- software_id	int
-- run_date	datetime
-- malware_detected	int

-- Expected output:
-- software_id	latest_run_count	difference_to_previous
-- 101	12	2

In [4]:
import pandas as pd

# Assuming you have a database connection and cursor set up
# Execute your SQL query to fetch the data
# For demonstration purposes, I'll create a sample dictionary with your data
data = {
    'software_id': [100, 100, 100, 101, 101, 101, 102],
    'run_date': [
        '2024-01-01 02:00:01',
        '2024-01-01 03:12:01',
        '2024-01-01 16:00:01',
        '2024-01-01 12:00:00',
        '2024-01-01 16:00:00',
        '2024-01-01 18:00:00',
        '2024-01-01 12:00:00'
    ],
    'malware_detected': [12, 15, 9, 9, 10, 12, 14]
}

# Create a DataFrame
malware = pd.DataFrame(data)

# Now you have a pandas DataFrame named 'malware'
print(malware)


   software_id             run_date  malware_detected
0          100  2024-01-01 02:00:01                12
1          100  2024-01-01 03:12:01                15
2          100  2024-01-01 16:00:01                 9
3          101  2024-01-01 12:00:00                 9
4          101  2024-01-01 16:00:00                10
5          101  2024-01-01 18:00:00                12
6          102  2024-01-01 12:00:00                14


In [5]:
query = '''
with cte as
(select * 
,dense_rank() over(partition by software_id order by run_date desc) as dr
,lag(malware_detected,1) over(partition by software_id order by run_date) as prev_malware
,count(*) over(partition by software_id) as run_count
from malware)

-- select * from cte
select software_id, malware_detected as latest_run_count,
malware_detected-prev_malware as difference_to_previous
from cte where dr=1 and run_count>1 and malware_detected>=10
'''

sqldf(query)

,software_id,latest_run_count,difference_to_previous
0,101,12,2


In [ ]:
-- CREATE TABLE malware (
--     software_id INT PRIMARY KEY,
--     run_date DATETIME,
--     malware_detected INT
-- );
-- INSERT INTO malware (software_id, run_date, malware_detected) VALUES
-- (100, '2024-01-01 02:00:01', 12),
-- (100, '2024-01-01 03:12:01', 15),
-- (100, '2024-01-01 16:00:01', 9),
-- (101, '2024-01-01 12:00:00', 9),
-- (101, '2024-01-01 16:00:00', 10),
-- (101, '2024-01-01 18:00:00', 12),
-- (102, '2024-01-01 12:00:00', 14);


In [ ]:
-- my solution1
with runs as
(   select * , count(*) as run_count
    from malware
    group by software_id
    having count(*)>=2
    )

, runs_rank as
(
    select * ,
    dense_rank() over(partition by software_id order by run_date desc) as dr,
    lead(malware_detected) over(partition by software_id order by run_date desc) as leading
    from malware
)

select rc.software_id, rc.malware_detected as latest_run_count, 
rc.malware_detected-leading as difference_to_previous
from runs r left join runs_rank rc on r.software_id=rc.software_id
where dr=1 and rc.malware_detected>=10;

In [ ]:
-- my solution2
with cte as
(select *,
lag(run_date,1) over(partition by software_id order by run_date) as lagging_date,
rank() over(partition by software_id order by run_date desc) as rnk
from malware)

-- select * from cte
select cte.software_id, cte.malware_detected as latest_run_count,
cte.malware_detected-m.malware_detected  as difference_to_previous
from cte join malware m on cte.lagging_date=m.run_date
where cte.malware_detected >=10 and cte.rnk=1


In [ ]:
-- provided solution
with cte1 as (
select *,
row_number() over(partition by software_id order by run_date desc) as rn
 from malware 
)
,cte2 as (
select software_id
, sum(case when rn=1 then malware_detected end) as latest_run_count
, sum(case when rn=2 then malware_detected end) as previous_run
from cte1
where rn in (1,2)
group by software_id
having count(*)=2
)
select software_id,latest_run_count
,(latest_run_count-previous_run) as difference_to_previous 
from cte2
where latest_run_count>=10

#### [22 - Highly Paid Employees](https://100daysofsql.com/new-courses/1/coding-test/23)

In [ ]:
-- You are given the data of employees along with their salary and department . Write an SQL to find list of employees who has salary greater than average employee salary of the company.  However , While calculating the company average salary to compare with an employee salary do not consider salaries of that employee's department.

 

-- Table: employee(primary key : emp_id)

-- column	datatype
-- emp_id	int
-- salary	date
-- department	varchar(20)
 


-- emp_id	salary	department
-- 102	50000	Analytics
-- 103	45000	Engineering
-- 104	48000	Engineering
-- 105	51000	Engineering
-- 106	46000	Science

In [6]:
import pandas as pd
data = {
    'emp_id': [100, 101, 102, 103, 104, 105, 106, 107, 108],
    'salary': [40000, 30000, 50000, 45000, 48000, 51000, 46000, 38000, 37000],
    'department': ['Analytics', 'Analytics', 'Analytics', 'Engineering', 'Engineering', 'Engineering', 'Science', 'Science', 'Science']
}

# Create a DataFrame
employee = pd.DataFrame(data)

# Now you have a pandas DataFrame named 'employee'
print(employee)

   emp_id  salary   department
0     100   40000    Analytics
1     101   30000    Analytics
2     102   50000    Analytics
3     103   45000  Engineering
4     104   48000  Engineering
5     105   51000  Engineering
6     106   46000      Science
7     107   38000      Science
8     108   37000      Science


In [7]:
query = '''
select * from employee e1
where salary > (select avg(salary) from employee e2 where e1.department!=e2.department)
'''

sqldf(query)

,emp_id,salary,department
0,102,50000,Analytics
1,103,45000,Engineering
2,104,48000,Engineering
3,105,51000,Engineering
4,106,46000,Science


In [ ]:
-- CREATE TABLE employee (
--     emp_id INT PRIMARY KEY,
--     salary INT,
--     department VARCHAR(20)
-- );
-- INSERT INTO employee (emp_id, salary, department) VALUES
-- (100, 40000, 'Analytics'),
-- (101, 30000, 'Analytics'),
-- (102, 50000, 'Analytics'),
-- (103, 45000, 'Engineering'),
-- (104, 48000, 'Engineering'),
-- (105, 51000, 'Engineering'),
-- (106, 46000, 'Science'),
-- (107, 38000, 'Science'),
-- (108, 37000, 'Science');


In [ ]:
select * from employee e1
where salary>(select avg(salary) from employee e2 where e1.department!=e2.department)

#### [23 - Warikoo 20-6-20](https://100daysofsql.com/new-courses/1/coding-test/24)

In [ ]:
-- Ankur Warikoo, an influential figure in Indian social media, shares a guideline in one of his videos called the 20-6-20 rule for determining whether one can afford to buy a phone. The rule entails three conditions:

-- Having enough savings to cover a 20 percent down payment.
-- Utilizing a maximum 6-month EMI plan (no-cost) for the remaining cost.
-- Ensuring that the monthly EMI does not exceed 20 percent of one's monthly salary.
-- Given the salary and savings of various users, along with data on phone costs, the task is to write an SQL to generate a list of phones (comma-separated) that each user can afford based on these criteria.

-- Table: users (primary key : user_name)

-- column name	datatype
-- user_name 	varchar(20)
-- monthly_salary 	int
-- savings 	int

-- Table: phones (primary key : phone_name)

-- column name	datatype
-- phone_name 	varchar(15)
-- cost 	int
 

-- Expected Output:

-- user_name	affordable_phones
-- Rahul	iphone-12,oneplus-12
-- Vivek	oneplus-12

In [ ]:
-- Create users table
CREATE TABLE users (
    user_name VARCHAR(20) PRIMARY KEY,
    monthly_salary INT,
    savings INT
);

-- Insert data into users table
INSERT INTO users (user_name, monthly_salary, savings) VALUES
('Rahul', 40000, 15000),
('Vivek', 70000, 10000);

-- Create phones table
CREATE TABLE phones (
    phone_name VARCHAR(15) PRIMARY KEY,
    cost INT
);

-- Insert data into phones table
INSERT INTO phones (phone_name, cost) VALUES
('iphone-12', 60000),
('oneplus-12', 50000),
('iphone-14', 70000);


In [ ]:
select user_name, group_concat(phone_name,",")  as affordable_phones
from phones, users
where savings>=cost*0.20  and monthly_salary*0.20>=cost*0.8/6
group by user_name;

#### [31 - Driver Ratings](https://100daysofsql.com/new-courses/1/coding-test/32)

In [ ]:
-- Suppose you are a data analyst working for a company that operates a ride-sharing platform similar to Uber. Your company is interested in analyzing the performance of drivers based on their ratings and wants to categorize them into different performance tiers. You are given a table named driver_ratings containing the following columns:

-- Table : driver_ratings (primary key : driver_id)

-- column name	datatype
-- driver_id	int
-- avg_rating	decimal(3,2)
 

-- Your task is to write an SQL query to categorize drivers equally into three performance tiers (top, middle, and bottom) based on their average ratings. Drivers with the highest average ratings should be placed in the top tier, drivers with ratings below the top tier but above the bottom tier should be placed in the middle tier, and drivers with the lowest average ratings should be placed in the bottom tier.

-- Expected Output:

-- driver_id	avg_rating	performance_tier
-- 7	4.9	Top
-- 1	4.8	Top
-- 5	4.7	Top
-- 12	4.6	Top
-- 2	4.5	Middle
-- 9	4.4	Middle
-- 4	4.2	Middle
-- 11	4.1	Middle
-- 3	3.9	Bottom
-- 8	3.8	Bottom
-- 6	3.6	Bottom
-- 10	3.5	Bottom
 

In [3]:
import pandas as pd
data=[
(1, 4.80),
(2, 4.50),
(3, 3.90),
(4, 4.20),
(5, 4.70),
(6, 3.60),
(7, 4.90),
(8, 3.80),
(9, 4.40),
(10, 3.50),
(11, 4.10),
(12, 4.60) 
]
cols=['driver_id', 'avg_rating']

driver_ratings=pd.DataFrame(data, columns=cols)
driver_ratings.head(2)

,driver_id,avg_rating
0,1,4.8
1,2,4.5


In [6]:
query = '''
with cte as
(select * 
, dense_rank() over(order by avg_rating desc) as dr
, count(*) over()/3 as group_ct
from driver_ratings)

select 
driver_id, avg_rating,
case when dr*1.0/group_ct<=1 then 'Top'
    when dr*1.0/group_ct<=2 then 'Middle'
    when dr*1.0/group_ct<=3 then 'Bottom' end as performance_tier
from cte
'''
sqldf(query)

,driver_id,avg_rating,performance_tier
0,7,4.9,Top
1,1,4.8,Top
2,5,4.7,Top
3,12,4.6,Top
4,2,4.5,Middle
5,9,4.4,Middle
6,4,4.2,Middle
7,11,4.1,Middle
8,3,3.9,Bottom
9,8,3.8,Bottom


In [ ]:
-- -- Create the driver_ratings table
-- CREATE TABLE driver_ratings (
--     driver_id INT PRIMARY KEY,
--     avg_rating DECIMAL(3,2)
-- );

-- -- Insert data into the driver_ratings table
-- INSERT INTO driver_ratings (driver_id, avg_rating) VALUES
-- (1, 4.80),
-- (2, 4.50),
-- (3, 3.90),
-- (4, 4.20),
-- (5, 4.70),
-- (6, 3.60),
-- (7, 4.90),
-- (8, 3.80),
-- (9, 4.40),
-- (10, 3.50),
-- (11, 4.10),
-- (12, 4.60);


In [ ]:
-- -- my sol
-- with cte2 as 
-- (select driver_id, row_number() over(order by avg_rating desc) as rankings 
-- from driver_ratings)


-- select 
-- dr.driver_id, avg_rating,
-- case when rankings*1.0/(select count(*)/3 from driver_ratings) <=1 then 'Top'
--     when rankings*1.0/(select count(*)/3 from driver_ratings) <=2 then 'Middle'
--     when rankings*1.0/(select count(*)/3 from driver_ratings) <=3 then 'Bottom' end as performance_tier
--  from driver_ratings dr join cte2 on dr.driver_id=cte2.driver_id
-- order by avg_rating desc


In [ ]:
-- --  provided sol
-- SELECT driver_id,avg_rating,
--     CASE
--         WHEN tier = 1 THEN 'Top'
--         WHEN tier = 2 THEN 'Middle'
--         ELSE 'Bottom'
--     END AS performance_tier
-- FROM (
--     SELECT
--         driver_id,avg_rating
--         ,NTILE(3) OVER (ORDER BY avg_rating DESC) AS tier
--     FROM
--         driver_ratings
-- ) AS ranked_drivers;

#### [32 - Customer Behavior](https://100daysofsql.com/new-courses/1/coding-test/33)

In [ ]:
-- Suppose you are a data analyst working for Zomato (a online food delivery company)  . Zomato is interested in analyzing customer food ordering behavior and wants to identify customers who have exhibited inconsistent patterns over time.

-- Your task is to write an SQL query to identify customers who have placed orders on both weekdays and weekends, but with a significant difference in the average order amount between weekdays and weekends. Specifically, you need to identify customers who have a minimum of 3 orders placed both on weekdays and weekends each, and where the average order amount on weekends is at least 20% higher than the average order amount on weekdays.

-- Your query should return the customer_id, the average order amount on weekends, the average order amount on weekdayss, and the percentage difference in average order amount between weekends and weekdays for each customer meeting the criteria.

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_id 	int
-- order_amount 	decimal(10,2)
 

-- Expected Output:

-- customer_id	weekend_avg_amount	weekday_avg_amount	percent_diff
-- 104	135.0	100.0	35.0
 

In [5]:
import pandas as pd

# Your data (replace with your actual data)
data = [
(1, 101, '2023-01-04', 100.00),
(2, 101, '2023-01-05', 200.00),
(3, 101, '2023-01-06', 80.00),
(4, 101, '2023-01-07', 110.00),
(5, 101, '2023-01-08', 90.00),
(6, 101, '2023-01-14', 130.00),
(7, 101, '2023-01-15', 150.00),
(8, 101, '2023-01-21', 100.00),
(9, 101, '2023-01-22', 120.00),
(10, 101, '2023-01-28', 90.00),
(11, 102, '2023-01-29', 110.00),
(12, 102, '2023-02-04', 80.00),
(13, 102, '2023-02-05', 100.00),
(14, 102, '2023-02-11', 120.00),
(15, 102, '2023-02-15', 140.00),
(16, 102, '2023-02-18', 80.00),
(17, 102, '2023-02-19', 100.00),
(18, 102, '2023-02-25', 120.00),
(19, 102, '2023-02-29', 140.00),
(20, 102, '2023-03-04', 80.00),
(31, 104, '2023-04-09', 140.00),
(32, 104, '2023-04-13', 80.00),
(33, 104, '2023-04-16', 100.00),
(34, 104, '2023-04-22', 200.00),
(35, 104, '2023-04-27', 140.00),
(36, 104, '2023-04-28', 80.00),
(37, 104, '2023-04-30', 100.00)
]

# Define column names
columns = ['order_id', 'customer_id', 'order_date', 'order_amount']

orders = pd.DataFrame(data, columns=columns)

display(orders.head())


,order_id,customer_id,order_date,order_amount
0,1,101,2023-01-04,100.0
1,2,101,2023-01-05,200.0
2,3,101,2023-01-06,80.0
3,4,101,2023-01-07,110.0
4,5,101,2023-01-08,90.0


In [3]:
# -- my sol
query = '''
with cte1 as 
(select *,
case when strftime('%w', order_date) in ('0','6') then 'weekend'
else 'weekday' end as day_type
from orders)

select customer_id
,round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
sum(case when day_type='weekend' then 1 else 0 end)) as weekend_avg_amount

,round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)) as weekday_avg_amount

,(round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
sum(case when day_type='weekend' then 1 else 0 end))-round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)))*100.0/
round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)) as percent_diff

from cte1 

group by customer_id

having sum(case when day_type='weekend' then 1 else 0 end)>=3 and
sum(case when day_type='weekday' then 1 else 0 end)>=3

order by percent_diff desc
limit 1

'''

sqldf(query)

,customer_id,weekend_avg_amount,weekday_avg_amount,percent_diff
0,104,135.0,100.0,35.0


In [ ]:
-- -- Create the orders table
-- CREATE TABLE orders (
--     order_id INT PRIMARY KEY,
--     order_date DATE,
--     customer_id INT,
--     order_amount DECIMAL(10,2)
-- );

-- -- Insert data into the orders table
-- INSERT INTO orders (order_id, customer_id, order_date, order_amount) VALUES
-- (1, 101, '2023-01-04', 100.00),
-- (2, 101, '2023-01-05', 200.00),
-- (3, 101, '2023-01-06', 80.00),
-- (4, 101, '2023-01-07', 110.00),
-- (5, 101, '2023-01-08', 90.00),
-- (6, 101, '2023-01-14', 130.00),
-- (7, 101, '2023-01-15', 150.00),
-- (8, 101, '2023-01-21', 100.00),
-- (9, 101, '2023-01-22', 120.00),
-- (10, 101, '2023-01-28', 90.00),
-- (11, 102, '2023-01-29', 110.00),
-- (12, 102, '2023-02-04', 80.00),
-- (13, 102, '2023-02-05', 100.00),
-- (14, 102, '2023-02-11', 120.00),
-- (15, 102, '2023-02-15', 140.00),
-- (16, 102, '2023-02-18', 80.00),
-- (17, 102, '2023-02-19', 100.00),
-- (18, 102, '2023-02-25', 120.00),
-- (19, 102, '2023-02-29', 140.00),
-- (20, 102, '2023-03-04', 80.00),
-- (31, 104, '2023-04-09', 140.00),
-- (32, 104, '2023-04-13', 80.00),
-- (33, 104, '2023-04-16', 100.00),
-- (34, 104, '2023-04-22', 200.00),
-- (35, 104, '2023-04-27', 140.00),
-- (36, 104, '2023-04-28', 80.00),
-- (37, 104, '2023-04-30', 100.00);


In [ ]:
-- -- my sol
-- with cte1 as 
-- (select *,
-- case when strftime('%w', order_date) in ('0','6') then 'weekend'
-- else 'weekday' end as day_type
-- from orders)

-- select customer_id
-- ,round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
-- sum(case when day_type='weekend' then 1 else 0 end)) as weekend_avg_amount
-- ,round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
-- sum(case when day_type='weekday' then 1 else 0 end)) as weekday_avg_amount

-- ,(round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
-- sum(case when day_type='weekend' then 1 else 0 end))-round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
-- sum(case when day_type='weekday' then 1 else 0 end)))*100.0/
-- round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
-- sum(case when day_type='weekday' then 1 else 0 end)) as percent_diff
-- from cte1 
-- group by customer_id
-- having sum(case when day_type='weekend' then 1 else 0 end)>=3 and
-- sum(case when day_type='weekday' then 1 else 0 end)>=3

-- order by percent_diff desc
-- limit 1


In [ ]:
-- -- provided sol

-- WITH order_summary AS (
--     SELECT
--         customer_id,
--         CASE
--             WHEN strftime('%w', order_date) IN ('0', '6') THEN 'Weekend'
--             ELSE 'Weekday'
--         END AS day_type,
--         COUNT(*) AS order_count,
--         AVG(order_amount) AS avg_order_amount
--     FROM
--         orders
--     GROUP BY
--         customer_id,
--         day_type
-- ),
-- weekend_weekday_summary AS (
--     SELECT
--         customer_id,
--         MAX(CASE WHEN day_type = 'Weekend' THEN order_count END) AS weekend_order_count,
--         MAX(CASE WHEN day_type = 'Weekday' THEN order_count END) AS weekday_order_count,
--         MAX(CASE WHEN day_type = 'Weekend' THEN avg_order_amount END) AS weekend_avg_amount,
--         MAX(CASE WHEN day_type = 'Weekday' THEN avg_order_amount END) AS weekday_avg_amount
--     FROM
--         order_summary
--     GROUP BY
--         customer_id
-- )
-- SELECT customer_id,weekend_avg_amount,weekday_avg_amount
-- ,(weekend_avg_amount - weekday_avg_amount)*100 / weekday_avg_amount AS percent_diff
-- FROM weekend_weekday_summary
-- where weekend_avg_amount>1.2*weekday_avg_amount

#### [33 - Points Table](https://100daysofsql.com/new-courses/1/coding-test/34)

In [ ]:
-- You are given table of cricket match played in a ICC cricket tounamant with the details of winner for each match. You need to derive a points table using below rules.

-- 1- For each win a team gets 2 points. 

-- 2- For a loss team gets 0 points.

-- 3- In case of a draw both the team gets 1 points each.

-- Table: icc_world_cup 

-- column name	datatype
-- team_1 	varchar(20)
-- team_2 	varchar(20)
-- winner 	varchar(20)


-- Expected Output:

-- team_name	match_played	no_of_wins	no_of_losses	total_points
-- Aus	2	0	1	1
-- Eng	3	1	1	3
-- India	2	2	0	4
-- NZ	1	1	0	2
-- SA	2	0	1	1
-- SL	2	0	1	1

In [8]:
data=[
 ('India', 'SL', 'India'),
('SL', 'Aus', 'Draw'),
('SA', 'Eng', 'Eng'),
('Eng', 'NZ', 'NZ'),
('Aus', 'India', 'India'),
('Eng', 'SA', 'Draw')   
]
columns=['team_1', 'team_2', 'winner']
icc_world_cup=pd.DataFrame(data, columns=columns)
icc_world_cup

,team_1,team_2,winner
0,India,SL,India
1,SL,Aus,Draw
2,SA,Eng,Eng
3,Eng,NZ,NZ
4,Aus,India,India
5,Eng,SA,Draw


In [9]:
query = '''
with all_matches as
(select team_1 as team , winner ,
case when winner = team_1 then 2 
when winner='Draw' then 1 else 0 end as points
from icc_world_cup
union all
select team_2 as team, winner,
case when winner = team_2 then 2 
when winner='Draw' then 1 else 0 end as points
 from icc_world_cup)

-- select * from all_matches

select 
team , count(*) as match_played,
sum(case when points=2 then 1 else 0 end) as no_of_wins,
sum(case when points=0 then 1 else 0 end) as no_of_losses,
sum(points) as total_points

from all_matches
group by team
'''

sqldf(query)

,team,match_played,no_of_wins,no_of_losses,total_points
0,Aus,2,0,1,1
1,Eng,3,1,1,3
2,India,2,2,0,4
3,NZ,1,1,0,2
4,SA,2,0,1,1
5,SL,2,0,1,1


In [ ]:
-- -- Create the icc_world_cup table
-- CREATE TABLE icc_world_cup (
--     team_1 VARCHAR(20),
--     team_2 VARCHAR(20),
--     winner VARCHAR(20)
-- );

-- -- Insert data into the icc_world_cup table
-- INSERT INTO icc_world_cup (team_1, team_2, winner) VALUES
-- ('India', 'SL', 'India'),
-- ('SL', 'Aus', 'Draw'),
-- ('SA', 'Eng', 'Eng'),
-- ('Eng', 'NZ', 'NZ'),
-- ('Aus', 'India', 'India'),
-- ('Eng', 'SA', 'Draw');


In [ ]:
-- -- my sol
-- with cte as
-- (
--     select team_1 as team_name, 
--     case when team_1 =winner then 1 else 0 end win_flag,
--     case when winner = 'Draw' then 1 else 0 end draw_flag
--     from icc_world_cup
--     union all 
--     select team_2 as team_name,
--     case when team_2 =winner then 1 else 0 end win_flag,
--     case when winner = 'Draw' then 1 else 0 end draw_flag
--     from icc_world_cup
-- )

-- select team_name,
-- count(*) as match_played,
-- sum(win_flag) as no_of_wins, count(*)-sum(win_flag)-sum(draw_flag) as no_of_losses,
-- sum(win_flag)*2+sum(draw_flag) as total_points

-- from cte
-- group by team_name

In [ ]:
-- -- provided sol
-- with cte as (
-- select team_1 as team_name
-- , case when team_1=winner then 1 else 0 end as win_flag
-- , case when winner='Draw' then 1 else 0 end as draw_flag
-- from icc_world_cup 
-- union all
-- select team_2 as team_name
-- , case when team_2=winner then 1 else 0 end as win_flag
-- , case when winner='Draw' then 1 else 0 end as draw_flag
-- from icc_world_cup 
-- )
-- select team_name,count(*) as match_played
-- ,sum(win_flag) as no_of_wins
-- ,count(*)-sum(win_flag)-sum(draw_flag) as no_of_losses
-- ,2*sum(win_flag)+sum(draw_flag) as total_points
-- from cte
-- group by team_name;

#### [34 - Customer Retention](https://100daysofsql.com/new-courses/1/coding-test/35)

In [ ]:
-- Customer retention can be defined as number of customers who continue to make purchases over a certain period compared to the total number of customers. Here's a step-by-step approach to calculate customer retention rate:

-- Determine the number of customers who made purchases in the current period (e.g., month : m )
-- Identify the number of customers from month m who made purchases in month m+1 , m+2 as well.
-- Suppose you are a data analyst working for an e-commerce company that sells various products online. The company is interested in measuring customer retention over the months to understand how many customers continue to make purchases over time. Your task is to write an SQL to derive customer retention month over month.

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_id 	int


-- Expected Output:

-- current_month	future_month	total_customers	retained_customers
-- 202312	202401	3	2
-- 202312	202402	3	1
-- 202401	202402	3	2
-- Explanation: For first row , in 202312 there were 3 distinct customers and out of them 2 did purchase in 202401 so they are retained customers. Similarly second row says that out of 3 customers in 202312 only 1 customer did purchase in 202402 and so on.

In [8]:
data=[
  (1, 101, '2023-12-01'),
(2, 102, '2023-12-02'),
(3, 103, '2023-12-13'),
(4, 104, '2024-01-05'),
(5, 101, '2024-01-06'),
(6, 102, '2024-01-08'),
(7, 102, '2024-01-10'),
(8, 104, '2024-02-15'),
(9, 105, '2024-02-20'),
(10, 102, '2024-02-25')  
]
cols=['order_id', 'customer_id', 'order_date']
orders = pd.DataFrame(data, columns=cols)
orders.head(2)

,order_id,customer_id,order_date
0,1,101,2023-12-01
1,2,102,2023-12-02


In [9]:
query = '''
with cte as (select 
distinct strftime('%Y%m', order_date) as yr_month , customer_id
from orders)

select 
    t1.yr_month as current_month, 
    count(distinct t1.customer_id) as total_customers,
    t2.yr_month as future_month, 
    count(distinct case when t1.customer_id= t2.customer_id then t1.customer_id end) as retained_customers
from 
    cte t1 join cte t2 on t1.yr_month<t2.yr_month
group by 
    t1.yr_month, t2.yr_month
'''
sqldf(query)

,current_month,total_customers,future_month,retained_customers
0,202312,3,202401,2
1,202312,3,202402,1
2,202401,3,202402,2


In [ ]:
-- -- Create the orders table
-- CREATE TABLE orders (
--     order_id INT PRIMARY KEY,
--     order_date DATE,
--     customer_id INT
-- );

-- -- Insert data into the orders table
-- INSERT INTO orders (order_id, customer_id, order_date) VALUES
-- (1, 101, '2023-12-01'),
-- (2, 102, '2023-12-02'),
-- (3, 103, '2023-12-13'),
-- (4, 104, '2024-01-05'),
-- (5, 101, '2024-01-06'),
-- (6, 102, '2024-01-08'),
-- (7, 102, '2024-01-10'),
-- (8, 104, '2024-02-15'),
-- (9, 105, '2024-02-20'),
-- (10, 102, '2024-02-25');


In [ ]:
-- -- # my sol
-- with cte as (select 
-- distinct strftime('%Y%m', order_date) as yr_month , customer_id
-- from orders)

-- select 
--     t1.yr_month as current_month, 
--     count(distinct t1.customer_id) as total_customers,
--     t2.yr_month as future_month, 
--     count(distinct case when t1.customer_id= t2.customer_id then t1.customer_id end) as retained_customers
-- from 
--     cte t1 join cte t2 on t1.yr_month<t2.yr_month
-- group by 
--     t1.yr_month, t2.yr_month

In [ ]:
-- -- provided sol
-- with cte as (select 
-- distinct strftime('%Y%m', order_date) as yr_month , customer_id
-- from orders)

-- select 
--     t1.yr_month as current_month, 
--     count(distinct t1.customer_id) as total_customers,
--     t2.yr_month as future_month, 
--     count(distinct case when t1.customer_id= t2.customer_id then t1.customer_id end) as retained_customers
-- from 
--     cte t1 join cte t2 on t1.yr_month<t2.yr_month
-- group by 
--     t1.yr_month, t2.yr_month

#### [35 - Order Fulfillment](https://100daysofsql.com/new-courses/1/coding-test/36)

In [ ]:
-- You are given two tables: products and orders. The products table contains information about each product, including the product ID and available quantity in the warehouse. The orders table contains details about customer orders, including the order ID, product ID, order date, and quantity requested by the customer.

-- Write an SQL query to generate a report listing the orders that can be fulfilled based on the available inventory in the warehouse, following a first-come-first-serve approach based on the order date. Each row in the report should include the order ID, product name, quantity requested by the customer, quantity actually fulfilled, and a comments column as below:

-- If the order can be completely fulfilled then 'Full Order',If the order can be partially fullfilled then 'Partial Order',If order can not be fulfilled at all then 'No Order'.

 

-- Table: products(primary key : product_id)

-- column name	datatype
-- product_id 	int
-- product_name 	varchar(10)
-- available_quantity 	int
 


-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- product_id 	int
-- order_date	date
-- quantity_requested 	int
 


-- Expected Output:

-- order_id	product_name	quantity_requested	qty_fulfilled	comments
-- 1	Product A	5	5	Full Order
-- 2	Product A	7	5	Partial Order
-- 3	Product B	10	10	Full Order
-- 4	Product B	10	10	Full Order
-- 5	Product B	5	0	No Order
-- 6	Product C	4	4	Full Order
-- 7	Product C	5	5	Full Order
-- 8	Product D	4	4	Full Order
-- 9	Product D	5	5	Full Order
-- 10	Product D	8	1	Partial Order
-- 11	Product D	5	0	No Order


In [10]:
import pandas as pd

# Data for the products table
products_data = {
    'product_id': [1, 2, 3, 4],
    'product_name': ['Product A', 'Product B', 'Product C', 'Product D'],
    'available_quantity': [10, 20, 15, 10]
}

# Create the products DataFrame
products = pd.DataFrame(products_data)

# Data for the orders table
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'product_id': [1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 4],
    'order_date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', 
                   '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11'],
    'quantity_requested': [5, 7, 10, 10, 5, 4, 5, 4, 5, 8, 5]
}

# Create the orders DataFrame
orders = pd.DataFrame(orders_data)

# Convert order_date to datetime format
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Display the DataFrames
print("Products DataFrame:")
display(products.head())
print("\nOrders DataFrame:")
display(orders.head())


Products DataFrame:


,product_id,product_name,available_quantity
0,1,Product A,10
1,2,Product B,20
2,3,Product C,15
3,4,Product D,10



Orders DataFrame:


,order_id,product_id,order_date,quantity_requested
0,1,1,2024-01-01,5
1,2,1,2024-01-02,7
2,3,2,2024-01-03,10
3,4,2,2024-01-04,10
4,5,2,2024-01-05,5


In [11]:
query = '''
with cte1 as
(select o.* , p.product_name as product_name,  p.available_quantity as available_quantity,
COALESCE(sum(quantity_requested) over(partition by o.product_id order by order_date rows between unbounded preceding and 1 preceding),0) as running_qty
from orders o join products p on o.product_id = p.product_id)
-- select * from cte1

select order_id, product_name, quantity_requested,  --available_quantity, running_qty,
case when quantity_requested <= (available_quantity-running_qty) then quantity_requested
    when (available_quantity <= running_qty) then 0
    else (available_quantity-running_qty) end as qty_fulfilled,

case when quantity_requested <= (available_quantity-running_qty) then 'Full Order'
    when (available_quantity <= running_qty) then 'No Order'
    else 'Partial Order' end as comments

from cte1
'''
sqldf(query)

,order_id,product_name,quantity_requested,qty_fulfilled,comments
0,1,Product A,5,5,Full Order
1,2,Product A,7,5,Partial Order
2,3,Product B,10,10,Full Order
3,4,Product B,10,10,Full Order
4,5,Product B,5,0,No Order
5,6,Product C,4,4,Full Order
6,7,Product C,5,5,Full Order
7,8,Product D,4,4,Full Order
8,9,Product D,5,5,Full Order
9,10,Product D,8,1,Partial Order


In [ ]:
-- -- Create the products table
-- CREATE TABLE products (
--     product_id INT PRIMARY KEY,
--     product_name VARCHAR(10),
--     available_quantity INT
-- );

-- -- Create the orders table
-- CREATE TABLE orders (
--     order_id INT PRIMARY KEY,
--     product_id INT,
--     order_date DATE,
--     quantity_requested INT,
--     FOREIGN KEY (product_id) REFERENCES products(product_id)
-- );

-- -- Insert data into the products table
-- INSERT INTO products (product_id, product_name, available_quantity) VALUES
-- (1, 'Product A', 10),
-- (2, 'Product B', 20),
-- (3, 'Product C', 15),
-- (4, 'Product D', 10);

-- -- Insert data into the orders table
-- INSERT INTO orders (order_id, product_id, order_date, quantity_requested) VALUES
-- (1, 1, '2024-01-01', 5),
-- (2, 1, '2024-01-02', 7),
-- (3, 2, '2024-01-03', 10),
-- (4, 2, '2024-01-04', 10),
-- (5, 2, '2024-01-05', 5),
-- (6, 3, '2024-01-06', 4),
-- (7, 3, '2024-01-07', 5),
-- (8, 4, '2024-01-08', 4),
-- (9, 4, '2024-01-09', 5),
-- (10, 4, '2024-01-10', 8),
-- (11, 4, '2024-01-11', 5);


In [ ]:
-- -- my sol
-- with cte1 as
-- (select o.* , p.product_name as product_name,  p.available_quantity as available_quantity,
-- COALESCE(sum(quantity_requested) over(partition by o.product_id order by order_date rows between unbounded preceding and 1 preceding),0) as running_qty
-- from orders o join products p on o.product_id = p.product_id)
-- -- select * from cte1

-- select order_id, product_name, quantity_requested,  --available_quantity, running_qty,
-- case when quantity_requested <= (available_quantity-running_qty) then quantity_requested
--     when (available_quantity <= running_qty) then 0
--     else (available_quantity-running_qty) end as qty_fulfilled,

-- case when quantity_requested <= (available_quantity-running_qty) then 'Full Order'
--     when (available_quantity <= running_qty) then 'No Order'
--     else 'Partial Order' end as comments

-- from cte1

In [ ]:
-- -- provided sol
-- with cte as (
-- select o.*
-- ,sum(quantity_requested) over(partition by o.product_id order by order_date) as running_requested_qty
-- ,p.available_quantity
-- ,p.product_name
-- from orders o
-- inner join products p on o.product_id=p.product_id
-- )
-- select order_id,product_name,quantity_requested
-- ,case when  running_requested_qty <= available_quantity then quantity_requested
-- when available_quantity-(running_requested_qty-quantity_requested) > 0 then available_quantity-(running_requested_qty-quantity_requested)
-- else 0 end as qty_fulfilled  
-- ,case when  running_requested_qty <= available_quantity then 'Full Order'
-- when available_quantity-(running_requested_qty-quantity_requested) > 0 then 'Partial Order'
-- else 'No Order' end as comments  
-- from cte;

#### [36 - Excess/insufficient Inventory](https://100daysofsql.com/new-courses/1/coding-test/37)

In [ ]:
-- Suppose you are a data analyst working for Flipkart. Your task is to identify excess and insuffienct inventory at various Flipkart warehouses in terms of no of units and cost. 

-- You are given an inventory table containing information about the inventory levels and required inventory target for each product and warehouse location combination.

-- Excess inventory is when inventory levels are greater than inventory targets else its insufficient inventory. Write an SQL to derive excess/insufficient Inventory for each location as well as at overall company level.

-- Table: inventory (primary key : location_id,product_id)

-- column name	datatype
-- location_id	int
-- product_id	int
-- inventory_level	int
-- inventory_target 	int
 

-- Table: products (primary key : product_id)

-- column name	datatype
-- product_id	int
-- unit_cost 	decimal(5,2)
 

-- Expected Output:

-- location_id	excess_insufficient_qty	excess_insufficient_value
-- 1	25	1347.5
-- 2	-25	-1420
-- 3	20	1180
-- 4	-12	-600
-- Overall	8	507.5

In [ ]:
-- Create the inventory table
CREATE TABLE inventory (
    location_id INT,
    product_id INT,
    inventory_level INT,
    inventory_target INT,
    PRIMARY KEY (location_id, product_id)
);

-- Create the products table
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    unit_cost DECIMAL(5, 2)
);

-- Insert data into the inventory table
INSERT INTO inventory (location_id, product_id, inventory_level, inventory_target) VALUES
(1, 101, 90, 80),
(1, 102, 100, 85),
(2, 102, 90, 80),
(2, 103, 70, 95),
(2, 104, 50, 60),
(3, 103, 120, 100),
(4, 104, 90, 102);

-- Insert data into the products table
INSERT INTO products (product_id, unit_cost) VALUES
(101, 51.5),
(102, 55.5),
(103, 59.0),
(104, 50.0);


In [ ]:
-- my sol

with cte as
(select i.*, (inventory_level-inventory_target) as excess_insufficient_qty,
(inventory_level-inventory_target)*p.unit_cost as excess_insufficient_value,
p.unit_cost
from inventory i left join products p on i.product_id=p.product_id)

select
    location_id, sum(excess_insufficient_qty) as excess_insufficient_qty, 
    sum(excess_insufficient_value) as excess_insufficient_value
from cte
group by location_id
union all
select 'Overall' as location_id, sum(excess_insufficient_qty) as excess_insufficient_qty,
sum(excess_insufficient_value) as excess_insufficient_value
from cte

In [ ]:
-- provided solution
with cte as (
select i.location_id
,sum(inventory_level-inventory_target) as excess_insufficient_qty
,sum((inventory_level-inventory_target)*p.unit_cost) as excess_insufficient_value 
from inventory i 
inner join products p on i.product_id=p.product_id
group by i.location_id)
select * from cte
union all
select 'Overall' as location_id 
, sum(excess_insufficient_qty) as excess_insufficient_qty
, sum(excess_insufficient_value) as excess_insufficient_value
from cte;

#### [37 - Zomato Membership](https://100daysofsql.com/new-courses/1/coding-test/38)

In [ ]:
-- Suppose you are working as a data analyst for Zomato . Zomato is planning to offer a premium membership to customers who have placed multiple orders in a single day.

-- Your task is to write a SQL to find those customers who have placed multiple orders in a single day atleast once , total order value generate by those customers and order value generated only by those orders. 

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	datetime
-- customer_name	varchar(20)
-- order_value	int
 

-- Expected Output:

-- customer_name	total_order_value	order_value
-- Mudit	780	550
-- Rahul	1300	1150

In [ ]:
-- Create the orders table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    order_date DATETIME,
    customer_name VARCHAR(20),
    order_value INT
);

-- Insert data into the orders table
INSERT INTO orders (order_id, order_date, customer_name, order_value) VALUES
(1, '2023-01-13 12:30:00', 'Rahul', 250),
(2, '2023-01-13 08:30:00', 'Rahul', 350),
(3, '2023-01-13 09:00:00', 'Mudit', 230),
(4, '2023-01-14 08:30:00', 'Rahul', 150),
(5, '2023-01-14 12:03:00', 'Suresh', 130),
(6, '2023-01-15 09:34:00', 'Mudit', 250),
(7, '2023-01-15 12:30:00', 'Mudit', 300),
(8, '2023-01-15 09:30:00', 'Rahul', 250),
(9, '2023-01-15 12:35:00', 'Rahul', 300),
(10, '2023-01-15 12:03:00', 'Suresh', 130);


In [ ]:
-- my sol
with cte as
(select  distinct customer_name, strftime('%Y%m%d', order_date) as order_day,count(*) as ct
from orders
group by strftime('%Y%m%d', order_date), customer_name
having count(*)>1)

select 
    o.customer_name, sum(order_value) as total_order_value,
    sum(case when ct is not Null then order_value else 0 end ) as order_value
from 
    orders o left join cte on o.customer_name=cte.customer_name 
    and strftime('%Y%m%d', order_date)=order_day
where o.customer_name in (select distinct customer_name from cte)
group by o.customer_name

In [ ]:
-- provided sol
with cte as (
select customer_name,strftime('%Y-%m-%d', order_date) as order_day
,count(*) as no_of_orders
 from orders 
group by customer_name,strftime('%Y-%m-%d', order_date)
having count(*)>1
)
select orders.customer_name,sum(orders.order_value) as total_order_value
,sum(case when cte.customer_name is not null then orders.order_value end) as order_value
from orders
left join cte on orders.customer_name=cte.customer_name and 
strftime('%Y-%m-%d', orders.order_date)=cte.order_day
where orders.customer_name in (select distinct customer_name from cte)
group by orders.customer_name;

#### [38 - Employees Inside Office (Part 1)](https://chatgpt.com/c/e7410c68-202c-4740-a634-92f530e627d0)

In [ ]:
-- A company record its employees movement In and Out of office in a table with 3 columns.

-- Table: employee_record

-- column name	datatype
-- emp_id	int
-- action	varchar(2)
-- created_at	datetime
 

-- Please note below points about the data:

-- 1- First entry for each employee is “in”

-- 2- Every “in” is succeeded by an “out”

-- 3- Employee can work across days

 

-- Write a SQL to find the number of employees inside the Office at “2019-04-01 19:05:00".

In [ ]:
-- Create the employee_record table
CREATE TABLE employee_record (
    emp_id INT,
    action VARCHAR(2),
    created_at DATETIME
);

-- Insert data into the employee_record table
INSERT INTO employee_record (emp_id, action, created_at) VALUES
(1, 'in', '2019-04-01 12:00:00'),
(1, 'out', '2019-04-01 15:00:00'),
(1, 'in', '2019-04-01 17:00:00'),
(1, 'out', '2019-04-01 21:00:00'),
(2, 'in', '2019-04-01 10:00:00'),
(2, 'out', '2019-04-01 16:00:00'),
(3, 'in', '2019-04-01 19:00:00'),
(3, 'out', '2019-04-02 05:00:00'),
(4, 'in', '2019-04-01 10:00:00'),
(4, 'out', '2019-04-01 20:00:00');


#### [39 - Employees Inside Office (Part 2)](https://100daysofsql.com/new-courses/1/coding-test/40)

In [ ]:
-- A company record its employees movement In and Out of office in a table with 3 columns.

-- Table: employee_record

-- column name	datatype
-- emp_id	int
-- action	varchar(2)
-- created_at	datetime
 


-- Please note below points about the data:

-- 1- First entry for each employee is “in”

-- 2- Every “in” is succeeded by an “out”

-- 3- Employee can work across days

 

-- Write an SQL to measure the amount of hours spent by each employee inside the office between “2019-04-01 14:00:00” and “2019-04-02 10:00:00"  in minutes. 

 

-- Expected Output:

-- emp_id	time_spent
-- 1	300.0
-- 2	0.0
-- 3	600.0
-- 4	960.0
-- 5	1200.0
-- 6	0.0

In [ ]:
-- Create the employee_record table
CREATE TABLE employee_record (
    emp_id INT,
    action VARCHAR(2),
    created_at DATETIME
);

-- Insert data into the employee_record table
INSERT INTO employee_record (emp_id, action, created_at) VALUES
(1, 'in', '2019-04-01 12:00:00'),
(1, 'out', '2019-04-01 15:00:00'),
(1, 'in', '2019-04-01 17:00:00'),
(1, 'out', '2019-04-01 21:00:00'),
(2, 'in', '2019-04-01 10:00:00'),
(2, 'out', '2019-04-01 13:00:00'),
(3, 'in', '2019-04-01 19:00:00'),
(3, 'out', '2019-04-02 05:00:00'),
(4, 'in', '2019-04-01 18:00:00'),
(4, 'out', '2019-04-02 20:00:00'),
(5, 'in', '2019-04-01 10:00:00'),
(5, 'out', '2019-04-02 11:00:00'),
(6, 'in', '2019-04-02 11:00:00'),
(6, 'out', '2019-04-02 16:00:00');


In [ ]:
with cte as 
(select * 
, lead(created_at) over(partition by emp_id order by created_at asc) as next_punch
from employee_record)

, cte2 as (select *,
    round((julianday(case when next_punch <'2019-04-01 14:00:00' then 0
        when created_at>'2019-04-02 10:00:00' then 0
        when next_punch<'2019-04-02 10:00:00' then next_punch 
        else '2019-04-02 10:00:00' end) -
    julianday(case when next_punch <'2019-04-01 14:00:00' then 0
        when created_at>'2019-04-02 10:00:00' then 0
        when created_at<'2019-04-01 14:00:00' then '2019-04-01 14:00:00' 
        else created_at end))*(24*60)) as time_spent
from cte
where 
action= 'in')

select 
emp_id, sum(time_spent) as time_spent
from cte2
group by emp_id

#### [40 - Credit Card Transactions (Part 1)](https://100daysofsql.com/new-courses/1/coding-test/41)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Your task is to find out highest spend card type and lowest spent card type for each city.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int
 

-- Expected Output:

-- city	highest_expense_type	lowest_expense_type
-- Bengaluru	Platinum	Silver
-- Delhi	Gold	Platinum
-- Mumbai	Platinum	Gold

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-13', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 1200),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 900),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 800),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-16', 'Platinum', 'F', 1900),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- my sol
with cte as 
(select city, card_type, sum(amount) as amount
from credit_card_transactions
group by city, card_type)

,cte2 as 
(select *,
max(amount) over(partition by city) as maxm,
min(amount) over(partition by city) as minm
from cte)

select city,
max(case when amount=maxm then card_type  end) as highest_expense_type,
max(case when amount=minm then card_type end)  as lowest_expense_type
from cte2
group by city

In [ ]:
-- provided sol

with cte as (
select city,card_type,sum(amount) as total_spend 
from credit_card_transactions
group by city,card_type
)
,cte2 as (
select *
,rank() over(partition by city order by total_spend desc) rn_high
,rank() over(partition by city order by total_spend) rn_low
from cte
)
select city
, max(case when rn_high=1 then card_type end) as highest_expense_type
, max(case when rn_low=1 then card_type end) as lowest_expense_type
from cte2
where rn_high=1 or rn_low=1
group by city;

#### [41 - Credit Card Transactions (Part 2)](https://100daysofsql.com/new-courses/1/coding-test/42)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Write an SQL to find percentage contribution of spends by females in each city.  Round the percentage to 2 decimal places.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int

-- Expected Output:

-- city	total_spend	female_spend	female_contribution
-- Bengaluru	3450	300	8.7
-- Delhi	1630	1430	87.73
-- Mumbai	4150	4150	100.0

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-13', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 1200),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 900),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 800),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-16', 'Platinum', 'F', 1900),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- # my sol

with cte as 
(select city, gender,
sum(amount) as amount
from credit_card_transactions
group by city, gender )

select city,
sum(amount) as total_spend,
sum(case when gender='F' then amount else 0 end) as female_spend,
round(sum(case when gender='F' then amount else 0 end)*100.0/sum(amount),2) as female_contribution

from cte
group by city

#### [42 - Credit Card Transactions (Part 3)](https://100daysofsql.com/new-courses/1/coding-test/43)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Write an SQL to find how many days each city took to reach cumulative spend of 1500 from its first day of transactions.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int
 

-- Expected Output:

-- city	first_transaction_date	tran_date_1500	no_of_days
-- Bengaluru	2024-01-12	2024-01-14	2.0
-- Delhi	2024-01-13	2024-01-16	3.0
-- Mumbai	2024-01-14	2024-01-18	4.0

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-12', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 300),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 500),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 700),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-18', 'Platinum', 'F', 1000),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- # my sol
with cte as 
(select * ,
sum(amount) over(partition by city order by transaction_date asc 
rows between unbounded preceding and current row) as cumm_amt,
min(transaction_date) over(partition by city order by transaction_date asc) as first_transaction

from credit_card_transactions)

-- select * from cte

select city
, first_transaction, transaction_date as tran_date_1500,
julianday(transaction_date)-julianday(first_transaction) as no_of_days
from
(select *,
row_number() over(partition by city order by cumm_amt) as rn
from cte 
where cumm_amt>=1500)
where rn=1

#### [43 - Adverse Reactions](https://100daysofsql.com/new-courses/1/coding-test/44)

In [ ]:
# problem statement
# In the field of pharmacovigilance, it's crucial to monitor and assess adverse reactions that patients may experience after taking certain medications. Adverse reactions, also known as side effects, can range from mild to severe and can impact the safety and efficacy of a medication.

# Consider a database schema for pharmacovigilance with the following tables:

# Table: patients (primary key : patient_id)

# column name	datatype
# patient_id 	int
# age	int
# name 	varchar(20)
# gender 	varchar(10)
 

# Table: medications (primary key : medication_id)

# column name	datatype
# medication_id 	int
# medication_name 	varchar(20)
# manufacturer 	varchar(20)
 

# Table: prescriptions (primary key : prescription_id)

# column name	datatype
# prescription_id 	int
# patient_id 	int
# medication_id 	int
# prescription_date 	date
 

# Table: adverse_reactions (primary key : reaction_id,patient_id)

# column name	datatype
# reaction_id	int
# patient_id	int
# reaction_description	varchar(20)
# reaction_date	date
 


# Your task is to write a SQL query to find the following:

# For each medication, count the number of adverse reactions reported within the first 30 days of the prescription being issued. Assume that the prescription date in the Prescriptions table represents the start date of the medication usage.

 

# Expected output:

# medication_name	manufacturer	num_adverse_reactions
# Medication A	Manufacturer X	4
# Medication B	Manufacturer Y	0
# Medication C	Manufacturer Z	1

In [14]:
import pandas as pd

# Creating the patients DataFrame
patients_data = {
    'patient_id': [1, 2, 3],
    'name': ['John Doe', 'Jane Smith', 'Alice Johnson'],
    'age': [35, 45, 25],
    'gender': ['Male', 'Female', 'Female']
}
patients = pd.DataFrame(patients_data)

# Creating the medications DataFrame
medications_data = {
    'medication_id': [1, 2, 3],
    'medication_name': ['Medication A', 'Medication B', 'Medication C'],
    'manufacturer': ['Manufacturer X', 'Manufacturer Y', 'Manufacturer Z']
}
medications = pd.DataFrame(medications_data)

# Creating the prescriptions DataFrame
prescriptions_data = {
    'prescription_id': [1, 2, 3, 4],
    'patient_id': [1, 1, 2, 3],
    'medication_id': [1, 2, 1, 3],
    'prescription_date': pd.to_datetime(['2023-01-01', '2023-02-15', '2023-03-10', '2023-04-20'])
}
prescriptions = pd.DataFrame(prescriptions_data)

# Creating the adverse_reactions DataFrame
adverse_reactions_data = {
    'reaction_id': [1, 2, 3, 4, 5],
    'patient_id': [1, 2, 3, 1, 2],
    'reaction_description': ['Nausea', 'Headache', 'Dizziness', 'Rash', 'Nausea'],
    'reaction_date': pd.to_datetime(['2023-01-05', '2023-03-20', '2023-05-01', '2023-01-20', '2023-04-05'])
}
adverse_reactions = pd.DataFrame(adverse_reactions_data)

# Displaying the DataFrames
# display("Patients DataFrame:")
# display(patients)
# display("Medications DataFrame:")
# display(medications)
# display("Prescriptions DataFrame:")
# display(prescriptions)
# display("Adverse Reactions DataFrame:")
# display(adverse_reactions)


In [15]:
## my sol
query = '''
with cte as 
(select * ,
julianday(reaction_date)-julianday(prescription_date) as reaction_time

from 
prescriptions p join adverse_reactions ar on p.patient_id = ar.patient_id
and p.prescription_date<ar.reaction_date
where julianday(reaction_date)-julianday(prescription_date) < 30)


select 
medication_name, manufacturer, 
sum(case when reaction_time>0 then 1 else 0 end) as num_adverse_reactions
from medications  left join cte on medications.medication_id=cte.medication_id
group by medication_name, manufacturer
'''

sqldf(query)

,medication_name,manufacturer,num_adverse_reactions
0,Medication A,Manufacturer X,4
1,Medication B,Manufacturer Y,0
2,Medication C,Manufacturer Z,1


#### [45 - Balanced Team](https://100daysofsql.com/new-courses/1/coding-test/46)

In [ ]:
# Suppose you are a manager of a data analytics company. You are tasked to build a new team consists of senior and junior data analysts. The total budget for the salaries in 70000 dollars.  You need to use the below criterian for hiring:

# 1- Keep hiring the seniors with the smallest salaries until you can not hire anymore seniors.

# 2- Use the remianing budget to hire the juniors with smallest salaries until you cannot hire anymore juniors.

# Sort the output by decreasing order of salary.

# Table: candidates (primary key : emp_id)

# column name	datatype
# emp_id 	int
# experience 	varchar(20)
# salary 	int
 

# Expected Output:

# emp_id	experience	salary
# 5	Senior	20000
# 4	Senior	16000
# 2	Junior	15000
# 1	Junior	10000

In [17]:
data = {
    'emp_id': [1, 2, 3, 4, 5, 6],
    'experience': ['Junior', 'Junior', 'Junior', 'Senior', 'Senior', 'Senior'],
    'salary': [10000, 15000, 40000, 16000, 20000, 50000]
}

# Create DataFrame
candidates = pd.DataFrame(data)
candidates

,emp_id,experience,salary
0,1,Junior,10000
1,2,Junior,15000
2,3,Junior,40000
3,4,Senior,16000
4,5,Senior,20000
5,6,Senior,50000


In [18]:
# my sol
query = '''
with cte as 
(select * 
, sum(salary) over(partition by experience order by salary asc) as cumm_salary
, 70000 as total_budget
from candidates
order by experience desc)

, seniors as 
(select emp_id , experience, salary
from cte 
where cumm_salary < total_budget and experience='Senior')

select * from seniors 
union all 
select emp_id , experience, salary from cte
where cumm_salary <= (select sum(salary) from seniors) and experience = 'Junior'
order by experience desc, salary desc
'''

sqldf(query)

,emp_id,experience,salary
0,5,Senior,20000
1,4,Senior,16000
2,2,Junior,15000
3,1,Junior,10000


#### [46 - Loan Repayment](https://100daysofsql.com/new-courses/1/coding-test/47)

In [ ]:
# You're working for a large financial institution that provides various types of loans to customers. Your task is to analyze loan repayment data to assess credit risk and improve risk management strategies.

 

# Write an SQL to create 2 flags for each loan as per below rules:

# fully_paid_flag: 1 if the loan was fully repaid irrespective of payment date else it should be 0.

# on_time_flag : 1 if the loan was fully repaid on or before due date else 0.

# Table: loans(primary key : loan_id)

# column name	datatype
# loan_id 	int
# customer_id 	int
# loan_amount 	int
# due_date 	date

# Table: payments(primary key : payment_id)

# column name	datatype
# payment_id 	int
# loan_id 	int
# amount_paid 	int
# payment_date 	date
 

# Expected Output:

# loan_id	loan_amount	due_date	fully_paid_flag	on_time_flag
# 1	5000	2023-01-15	0	0
# 2	8000	2023-02-20	1	1
# 3	10000	2023-03-10	0	0
# 4	6000	2023-04-05	1	1
# 5	7000	2023-05-01	1	0

In [60]:
import pandas as pd

# Define the data for the loans table
loans_data = {
    'loan_id': [1, 2, 3, 4, 5],
    'customer_id': [1, 2, 3, 4, 5],
    'loan_amount': [5000, 8000, 10000, 6000, 7000],
    'due_date': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-01']
}

# Create the DataFrame for loans
loans = pd.DataFrame(loans_data)

# Convert 'due_date' to datetime
loans['due_date'] = pd.to_datetime(loans['due_date'])

# Define the data for the payments table
payments_data = {
    'payment_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'loan_id': [1, 1, 2, 3, 4, 4, 5, 5],
    'payment_date': [
        '2023-01-10', '2023-02-10', '2023-02-20', '2023-04-20', 
        '2023-03-15', '2023-04-02', '2023-04-02', '2023-05-02'
    ],
    'amount_paid': [2000, 1500, 8000, 5000, 2000, 4000, 4000, 3000]
}

# Create the DataFrame for payments
payments = pd.DataFrame(payments_data)

# Convert 'payment_date' to datetime
payments['payment_date'] = pd.to_datetime(payments['payment_date'])

# Display the DataFrames
print("Loans DataFrame:")
print(loans)
print("\nPayments DataFrame:")
print(payments)


Loans DataFrame:
   loan_id  customer_id  loan_amount   due_date
0        1            1         5000 2023-01-15
1        2            2         8000 2023-02-20
2        3            3        10000 2023-03-10
3        4            4         6000 2023-04-05
4        5            5         7000 2023-05-01

Payments DataFrame:
   payment_id  loan_id payment_date  amount_paid
0           1        1   2023-01-10         2000
1           2        1   2023-02-10         1500
2           3        2   2023-02-20         8000
3           4        3   2023-04-20         5000
4           5        4   2023-03-15         2000
5           6        4   2023-04-02         4000
6           7        5   2023-04-02         4000
7           8        5   2023-05-02         3000


In [61]:
query = '''
SELECT l.loan_id, loan_amount, due_date ,
case when loan_amount=sum(amount_paid) then 1 else 0 end as fully_paid_flag
,case when max(payment_date)<=due_date then 1 else 0 end as on_time_flag
from loans l left join payments p on
l.loan_id=p.loan_id
group by l.loan_id
'''
sqldf(query)

,loan_id,loan_amount,due_date,fully_paid_flag,on_time_flag
0,1,5000,2023-01-15 00:00:00.000000,0,0
1,2,8000,2023-02-20 00:00:00.000000,1,1
2,3,10000,2023-03-10 00:00:00.000000,0,0
3,4,6000,2023-04-05 00:00:00.000000,1,1
4,5,7000,2023-05-01 00:00:00.000000,1,0


#### [47 - LinkedIn Algorithm](https://100daysofsql.com/new-courses/1/coding-test/48)

In [ ]:
# LinkedIn stores information of post likes in below format . So everytime a user likes a post there will be an entry made in post_likes table.

# Table : post_likes (primary key : user_id,post_id)

# column name	datatype
# user_id	int
# post_id	int
 


# LinkedIn also stores the information when someone follows another user in below format

# Table : user_follows (primary key : user_id ,follows_user_id )

# column name	datatype
# user_id	int
# follows_user_id	int
 


# The marketing team wants to send one recommendation post to each user . Write an SQL to find out that one post_id for each user that is liked by the most number of users that they follow.
# Please note that team do not want to recommend a post which is already liked by the user.

# If for any user,  2 or more posts are liked by equal number of users that they follow then select the smallest post id.

 

# Expected Output:

# user_id	post_id	no_of_likes
# 1	200	1
# 2	400	3
# 3	100	2
# 4	100	2

In [62]:
import pandas as pd

# Define the data for the post_likes table
post_likes_data = {
    'user_id': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1],
    'post_id': [100, 100, 200, 300, 300, 300, 300, 400, 400, 400, 400]
}

# Create the DataFrame for post_likes
post_likes = pd.DataFrame(post_likes_data)

# Define the data for the user_follows table
user_follows_data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4],
    'follows_user_id': [2, 3, 4, 1, 3, 4, 1, 2, 1, 2]
}

# Create the DataFrame for user_follows
user_follows = pd.DataFrame(user_follows_data)

# Display the DataFrames
print("Post Likes DataFrame:")
display(post_likes)
print("\nUser Follows DataFrame:")
display(user_follows)


Post Likes DataFrame:


,user_id,post_id
0,1,100
1,2,100
2,3,200
3,4,300
4,5,300
5,1,300
6,2,300
7,3,400
8,4,400
9,5,400



User Follows DataFrame:


,user_id,follows_user_id
0,1,2
1,1,3
2,1,4
3,2,1
4,2,3
5,2,4
6,3,1
7,3,2
8,4,1
9,4,2


#### [51 - Dashboard Visits](https://100daysofsql.com/new-courses/1/coding-test/52)

In [ ]:
# You're working as a data analyst for a popular website's dashboard analytics team. Your task is to analyze user visits to the dashboard and identify users who are highly engaged with the platform. The dashboard records user visits along with timestamps to provide insights into user activity patterns.

# A user can visit the dashboard multiple times within a day. However, to be counted as separate visits, there should be a minimum gap of 60 minutes between consecutive visits. If the next visit occurs within 60 minutes of the previous one, it's considered part of the same visit.

# Table: dashboard_visit(primary key : order_id)

# column name	datatype
# user_id 	int
# visit_time 	datetime
 

# Write an SQL query to find total number of visits by each user along with number of distinct days user has visited the dashboard. While calculating the number of distinct days you have to consider a visit even if it is same as previous days visit.

# So for example if there is a visit at 2024-01-12 23:30:00 and next visit at 2024-01-13 00:15:00 , The visit on 13th will not be considered as new visit because it is within 1 hour window of previous visit but number of days will be counted as 2 only.

 

# Expected Output:

# user_id	no_of_visits	visit_days
# U1	2	1
# U2	2	2
# U3	1	1
# U4	3	4

In [2]:
# import pandas as pd
# from datetime import datetime

# Define the data
data = {
    'user_id': ['U1', 'U1', 'U1', 'U2', 'U2', 'U3', 'U3', 'U4', 'U4', 'U4', 'U4'],
    'visit_time': [
        '2021-12-04 10:44:56', '2021-12-04 10:55:56', '2021-12-04 12:56:56',
        '2021-12-05 12:55:50', '2021-12-06 14:55:50', '2021-11-06 17:53:50',
        '2021-11-06 17:56:50', '2021-11-29 13:53:50', '2021-12-01 10:53:50',
        '2021-12-06 23:53:50', '2021-12-07 00:20:50'
    ]
}

# Create the DataFrame
dashboard_visit = pd.DataFrame(data)

# Convert 'visit_time' to datetime
dashboard_visit['visit_time'] = pd.to_datetime(dashboard_visit['visit_time'])

# Display the DataFrame
dashboard_visit


,user_id,visit_time
0,U1,2021-12-04 10:44:56
1,U1,2021-12-04 10:55:56
2,U1,2021-12-04 12:56:56
3,U2,2021-12-05 12:55:50
4,U2,2021-12-06 14:55:50
5,U3,2021-11-06 17:53:50
6,U3,2021-11-06 17:56:50
7,U4,2021-11-29 13:53:50
8,U4,2021-12-01 10:53:50
9,U4,2021-12-06 23:53:50


In [6]:
# my sol-1
query='''
with cte as 
(select *,
lag(visit_time) over(partition by user_id order by visit_time) as prev_visit
from dashboard_visit)

select user_id,
sum(case when (julianday(visit_time)-julianday(prev_visit))*24*60>=60 or prev_visit is null then 1 else 0 end) as no_of_visits
, count(distinct strftime('%Y%m%d', visit_time)) as visit_days
from cte 
group by user_id
'''
sqldf(query)

,user_id,no_of_visits,visit_days
0,U1,2,1
1,U2,2,2
2,U3,1,1
3,U4,3,4


In [7]:
# my sol-2
query='''
with cte as 
(select * ,
coalesce((julianday(visit_time) - julianday(lag(visit_time) over(partition by user_id)))*24*60,0) as visit_diff

from dashboard_visit)


-- select * from cte;

select 
user_id, 
sum(case when visit_diff>=60 or visit_diff=0 then 1 else 0 end) as no_of_visits,
count(distinct strftime('%Y%m%d', visit_time)) as visit_days
from cte 
group by user_id
'''
sqldf(query)

,user_id,no_of_visits,visit_days
0,U1,2,1
1,U2,2,2
2,U3,1,1
3,U4,3,4


In [8]:
# provided sol
query='''
WITH previous_visits AS (
    SELECT 
        user_id,
        visit_time,
        LAG(visit_time) OVER (PARTITION BY user_id ORDER BY visit_time) AS previous_visit_time
    FROM
        dashboard_visit
)
, vigit_flag as (
select user_id, previous_visit_time,visit_time
,CASE WHEN previous_visit_time IS NULL THEN 1
      WHEN (julianday(visit_time) - julianday(previous_visit_time))*24>1 THEN 1
            ELSE 0
        END AS new_visit_flag
 from previous_visits
)
select user_id, sum(new_visit_flag) as no_of_visits, count(distinct strftime('%d-%m-%Y', visit_time)) as visit_days
from vigit_flag
group by user_id;
'''
sqldf(query)

,user_id,no_of_visits,visit_days
0,U1,2,1
1,U2,2,2
2,U3,1,1
3,U4,3,4


#### [52 - Final Balance](https://100daysofsql.com/new-courses/1/coding-test/53)

In [ ]:
# You are given history of your bank account for the year 2020 . Each transaction was wither a credit card payment or incoming transfer. There is a fee of holding a credit card which you have to pay every month,  Fee is 5 per month. 

# However you are not charged for a given month if you made atleast 2 credit card payments for a cost of atleast 100 within that month. Note that this fee is not included in the supplied history of transactions.

# You are given transactions with the follwing structure:
# Table : Transactions 
# Columns : amount int , transaction_date date

 

# Each row in the table contains information about a single transaction. If the amount value is negative , it is a credit card payment otherwise it is an incoming transfer.

# At the beginning of the year, the balance of your account was 0 . Your task is to compute the balance at the end of the year. 

# Expected Output:

# final_balance
# -95

In [12]:
# import pandas as pd

# Define the data
data = {
    'transaction_date': [
        '2020-01-01', '2020-01-05', '2020-01-24', '2020-03-01', '2020-03-01',
        '2020-04-10', '2020-04-13', '2020-07-05', '2020-10-19', '2020-12-01',
        '2020-12-05', '2020-12-19'
    ],
    'amount': [-30, -80, 30, -40, 30, 70, 40, -30, 60, -40, -30, -20]
}

# Create the DataFrame
transactions = pd.DataFrame(data)

# Convert 'transaction_date' to datetime
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

# Display the DataFrame
print(transactions)


   transaction_date  amount
0        2020-01-01     -30
1        2020-01-05     -80
2        2020-01-24      30
3        2020-03-01     -40
4        2020-03-01      30
5        2020-04-10      70
6        2020-04-13      40
7        2020-07-05     -30
8        2020-10-19      60
9        2020-12-01     -40
10       2020-12-05     -30
11       2020-12-19     -20


In [11]:
# my sol

In [13]:
# provided sol
query = '''
with cte as (
select strftime('%Y%m', transaction_date) as tran_month,amount
 from transactions
)
,cte2 as (
select tran_month
, sum(amount) as net_amount , sum(case when amount<0 then -1*amount else 0 end) as credit_card_amount
, sum(case when amount<0 then 1 else 0 end) as credit_card_transact_cnt
from cte 
group by tran_month
)
select sum(net_amount) - sum(case when credit_card_amount >=100 and credit_card_transact_cnt >=2 then 0 else 5 end)
- 5*(12-(select count(distinct tran_month) from cte)) as final_balance
from cte2;
'''
sqldf(query)

,final_balance
0,-95


#### [53 - Lead Time](https://100daysofsql.com/new-courses/1/coding-test/54)

In [14]:
# import pandas as pd

# Define the data for the orders table
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': [
        '2024-03-14', '2024-03-10', '2024-03-04', 
        '2024-03-05', '2024-03-03', '2024-03-07'
    ],
    'ship_date': [
        '2024-03-20', '2024-03-16', '2024-03-12', 
        '2024-03-07', '2024-03-08', '2024-03-24'
    ]
}

# Create the DataFrame for orders
orders = pd.DataFrame(orders_data)

# Convert 'order_date' and 'ship_date' to datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['ship_date'] = pd.to_datetime(orders['ship_date'])

# Define the data for the holidays table
holidays_data = {
    'holiday_id': [1, 2, 3],
    'holiday_date': ['2024-03-10', '2024-03-18', '2024-03-21']
}

# Create the DataFrame for holidays
holidays = pd.DataFrame(holidays_data)

# Convert 'holiday_date' to datetime
holidays['holiday_date'] = pd.to_datetime(holidays['holiday_date'])

# Display the DataFrames
print("Orders DataFrame:")
display(orders)
print("\nHolidays DataFrame:")
display(holidays)


Orders DataFrame:


,order_id,order_date,ship_date
0,1,2024-03-14,2024-03-20
1,2,2024-03-10,2024-03-16
2,3,2024-03-04,2024-03-12
3,4,2024-03-05,2024-03-07
4,5,2024-03-03,2024-03-08
5,6,2024-03-07,2024-03-24



Holidays DataFrame:


,holiday_id,holiday_date
0,1,2024-03-10
1,2,2024-03-18
2,3,2024-03-21


In [16]:
query = '''
with cte as 
(select  order_id,
case when strftime('%w', order_date) = '0' then date(order_date, '+1 day')
    when strftime('%w', order_date) = '6' then date(order_date, '+2 day')
    else order_date end as new_order_date
,case when strftime('%w', ship_date) = '0' then date(ship_date, '-2 day')
    when strftime('%w', ship_date) = '6' then date(ship_date, '-1 day')
    else ship_date end as new_ship_date
from orders)

-- select * from cte

select order_id,
(julianday(new_ship_date)-julianday(new_order_date))-
sum(case when holiday_date is null then 0 
    else 1 end ) as lead_time
from cte left join holidays h 
on h.holiday_date between new_order_date and new_ship_date
group by order_id

'''
qdf=sqldf(query)

In [19]:
qdf.columns

Index(['order_id', 'lead_time'], dtype='object')

#### [54 - Instagram Marketing Agency](https://100daysofsql.com/new-courses/1/coding-test/55)

In [ ]:
# You are working for a marketing agency that manages multiple Instagram influencer accounts. Your task is to analyze the engagement performance of these influencers before and after they join your company.

# Write an SQL query to calculate average engagement growth rate percent for each influencer after they joined your company compare to before. Round the growth rate to 2 decimal places and sort the output in descreasing order of growth rate.

# Engagement = # of likes + # of comments on each post

# Table: influencers (primary key : influencer_id )

# column name	datatype
# influencer_id 	int
# username 	varchar(20)
# join_date 	date
 

# Table: posts (primary key : post_id)

# column name	datatype
# post_id 	int
# influencer_id 	int
# post_date 	date
# likes 	int
# comments 	int
 


# Expected output:

# username	before_engagement	after_engagement	growth
# Ankit	150.0	195.0	30.0
# Rahul	202.5	250.0	23.46

In [20]:
# import pandas as pd

# Define the data for the influencers table
influencers_data = {
    'influencer_id': [1, 2, 3],
    'username': ['Ankit', 'Rahul', 'Suresh'],
    'join_date': ['2023-02-01', '2023-03-05', '2023-05-20']
}

# Create the DataFrame for influencers
influencers = pd.DataFrame(influencers_data)

# Convert 'join_date' to datetime
influencers['join_date'] = pd.to_datetime(influencers['join_date'])

# Define the data for the posts table
posts_data = {
    'post_id': [1, 2, 3, 4, 1, 2, 3, 4],
    'influencer_id': [1, 1, 1, 1, 2, 2, 2, 2],
    'post_date': [
        '2023-01-05', '2023-01-10', '2023-02-05', '2023-02-10',
        '2023-02-15', '2023-02-20', '2023-03-10', '2023-03-15'
    ],
    'likes': [100, 150, 200, 120, 150, 200, 250, 200],
    'comments': [20, 30, 45, 25, 30, 25, 15, 35]
}

# Create the DataFrame for posts
posts = pd.DataFrame(posts_data)

# Convert 'post_date' to datetime
posts['post_date'] = pd.to_datetime(posts['post_date'])

# Display the DataFrames
print("Influencers DataFrame:")
display(influencers)
print("\nPosts DataFrame:")
display(posts)


Influencers DataFrame:


,influencer_id,username,join_date
0,1,Ankit,2023-02-01
1,2,Rahul,2023-03-05
2,3,Suresh,2023-05-20



Posts DataFrame:


,post_id,influencer_id,post_date,likes,comments
0,1,1,2023-01-05,100,20
1,2,1,2023-01-10,150,30
2,3,1,2023-02-05,200,45
3,4,1,2023-02-10,120,25
4,1,2,2023-02-15,150,30
5,2,2,2023-02-20,200,25
6,3,2,2023-03-10,250,15
7,4,2,2023-03-15,200,35


In [21]:
#  my sol
query = '''
with cte as 
(select 
username , 
avg(case when post_date < join_date then likes+comments end) as before_engagement
,avg(case when post_date >= join_date then likes+comments end) as after_engagement
from influencers i join posts p on 
i.influencer_id=p.influencer_id
group by username )

select *,
round((after_engagement-before_engagement)*100.0/before_engagement,2) growth
from cte
'''
sqldf(query)

,username,before_engagement,after_engagement,growth
0,Ankit,150.0,195.0,30.00
1,Rahul,202.5,250.0,23.46


In [22]:
# provided sol
query = '''
with cte as (
select  i.username 
, avg(case when p.post_date < i.join_date then likes+comments end) as before_engagement
, avg(case when p.post_date > i.join_date then likes+comments end) as after_engagement
from posts p
inner join influencers i on i.influencer_id=p.influencer_id
group by i.username 
)
select *
,round((after_engagement-before_engagement)*100/before_engagement,2) as growth
from cte
order by growth desc;
'''
sqldf(query)

,username,before_engagement,after_engagement,growth
0,Ankit,150.0,195.0,30.00
1,Rahul,202.5,250.0,23.46


#### [55 - Data Cleaning](https://100daysofsql.com/new-courses/1/coding-test/56)

In [ ]:
# You are a data analyst working for a transportation company that manages routes between various cities. The company stores information about distances between cities in a table named city_distances. Each record in the table represents the distance between a source city and a destination city.

# However, due to data entry errors or inconsistencies, there are duplicate records in the table where the source and destination cities are interchanged. These duplicates need to be removed to ensure data integrity and accuracy. In case the distance value in the duplicate records is not same then take average of the 2 distances.

# Sort the ouput in descending order by distance.

# Table : city_distances

# source	destination	distance
# Bangalore	Hyderbad	350
# Hyderbad	Bangalore	350
# Mumbai	Delhi	400
# Delhi	Mumbai	450
# Chennai	Pune	500
 

# Expected Output:

# source	destination	distance
# Chennai	Pune	500.0
# Delhi	Mumbai	425.0
# Bangalore	Hyderbad	350.0

In [25]:
import pandas as pd

# Define the data
data = {
    'source': ['Bangalore', 'Hyderbad', 'Mumbai', 'Delhi', 'Chennai'],
    'destination': ['Hyderbad', 'Bangalore', 'Delhi', 'Mumbai', 'Pune'],
    'distance': [350, 350, 400, 450, 500]
}

# Create the DataFrame
city_distances = pd.DataFrame(data)

# Display the DataFrame
print(city_distances)


      source destination  distance
0  Bangalore    Hyderbad       350
1   Hyderbad   Bangalore       350
2     Mumbai       Delhi       400
3      Delhi      Mumbai       450
4    Chennai        Pune       500


In [27]:
# my sol
query = '''
with cte as
(select 
case when source>destination then destination else source end as source
,case when source<destination then destination else source end as destination
, distance
from city_distances)

select source, destination, avg(distance) as distance
from cte
group by source, destination
order by distance desc
'''
sqldf(query)

,source,destination,distance
0,Chennai,Pune,500.0
1,Delhi,Mumbai,425.0
2,Bangalore,Hyderbad,350.0


In [30]:
# # provided sol

# query = '''
# with cte as 
#  ( select case when sourcesource then destination else source end as destination,distance from city_distances ) 
#  select source,destination,avg(distance) as distance from cte group by source,destination order by distance desc
# '''
# sqldf(query)

#### [56 - Most Visited Floor](https://100daysofsql.com/new-courses/1/coding-test/57)

In [ ]:
# You are a facilities manager at a corporate office building, responsible for tracking employee visits, floor preferences, and resource usage within the premises. The office building has multiple floors, each equipped with various resources such as desks, computers, monitors, and other office supplies.

# You have a database table named entries that stores information about employee visits to the office building. Each record in the table represents a visit by an employee and includes details such as their name, the floor they visited, and the resources they used during their visit.

# Input Table: entries:

# emp_name	address	floor	resources
# Ankit	Bangalore	1	CPU
# Ankit	Bangalore	1	CPU
# Ankit	Bangalore	2	DESKTOP
# Bikaas	Bangalore	2	DESKTOP
# Bikaas	Bangalore	2	DESKTOP
# Bikaas	Bangalore	1	MONITOR
 

# Write an SQL query to retrieve the total visits, most visited floor, and resources used by each employee.

# Expected Output:

# emp_name	total_visits	most_visited_floor	resources_used
# Ankit	3	1	CPU,DESKTOP
# Bikaas	3	2	DESKTOP,MONITOR

In [31]:
import pandas as pd

# Define the data
data = {
    'emp_name': ['Ankit', 'Ankit', 'Ankit', 'Bikaas', 'Bikaas', 'Bikaas'],
    'address': ['Bangalore', 'Bangalore', 'Bangalore', 'Bangalore', 'Bangalore', 'Bangalore'],
    'floor': [1, 1, 2, 2, 2, 1],
    'resources': ['CPU', 'CPU', 'DESKTOP', 'DESKTOP', 'DESKTOP', 'MONITOR']
}

# Create the DataFrame
entries = pd.DataFrame(data)

# Display the DataFrame
print(entries)


  emp_name    address  floor resources
0    Ankit  Bangalore      1       CPU
1    Ankit  Bangalore      1       CPU
2    Ankit  Bangalore      2   DESKTOP
3   Bikaas  Bangalore      2   DESKTOP
4   Bikaas  Bangalore      2   DESKTOP
5   Bikaas  Bangalore      1   MONITOR


In [32]:
# my sol

query = '''
with cte as
(select 
emp_name, floor, count(*) as floor_freq,
row_number() over(partition by emp_name order by count(*) desc) as rn,
group_concat(distinct resources) as resources
from entries
group by emp_name, floor)

select 
emp_name, sum(floor_freq) as total_visits, floor as most_visited_floor,
group_concat(distinct resources) as resources_used
from cte
group by emp_name
'''
sqldf(query)

,emp_name,total_visits,most_visited_floor,resources_used
0,Ankit,3,1,"CPU,DESKTOP"
1,Bikaas,3,2,"DESKTOP,MONITOR"


In [33]:
# provided sol

query = '''
with cte_resources as (
select emp_name ,count(*) as total_visits ,group_concat(distinct resources) as resources_used
from entries
group by emp_name
),
floor_visits as (
select emp_name,floor,count(*) as no_of_floor_visits
,row_number() over(partition by emp_name order by count(*) desc) as rn
from entries
group by emp_name,floor
)
select fv.emp_name,r.total_visits,fv.floor as most_visited_floor,r.resources_used
 from floor_visits fv 
 inner join cte_resources r on fv.emp_name=r.emp_name
 where rn=1
'''
sqldf(query)

,emp_name,total_visits,most_visited_floor,resources_used
0,Ankit,3,1,"CPU,DESKTOP"
1,Bikaas,3,2,"DESKTOP,MONITOR"


#### [56 - Most Visited Floor](https://100daysofsql.com/new-courses/1/coding-test/57)

In [ ]:
# You are a facilities manager at a corporate office building, responsible for tracking employee visits, floor preferences, and resource usage within the premises. The office building has multiple floors, each equipped with various resources such as desks, computers, monitors, and other office supplies.

# You have a database table named entries that stores information about employee visits to the office building. Each record in the table represents a visit by an employee and includes details such as their name, the floor they visited, and the resources they used during their visit.

# Input Table: entries:

# emp_name	address	floor	resources
# Ankit	Bangalore	1	CPU
# Ankit	Bangalore	1	CPU
# Ankit	Bangalore	2	DESKTOP
# Bikaas	Bangalore	2	DESKTOP
# Bikaas	Bangalore	2	DESKTOP
# Bikaas	Bangalore	1	MONITOR
 

# Write an SQL query to retrieve the total visits, most visited floor, and resources used by each employee.

# Expected Output:

# emp_name	total_visits	most_visited_floor	resources_used
# Ankit	3	1	CPU,DESKTOP
# Bikaas	3	2	DESKTOP,MONITOR

In [55]:
import pandas as pd

# Define the data
data = {
    'emp_name': ['Ankit', 'Ankit', 'Ankit', 'Bikaas', 'Bikaas', 'Bikaas'],
    'address': ['Bangalore', 'Bangalore', 'Bangalore', 'Bangalore', 'Bangalore', 'Bangalore'],
    'floor': [1, 1, 2, 2, 2, 1],
    'resources': ['CPU', 'CPU', 'DESKTOP', 'DESKTOP', 'DESKTOP', 'MONITOR']
}

# Create the DataFrame
entries = pd.DataFrame(data)

# Display the DataFrame
print(entries)


  emp_name    address  floor resources
0    Ankit  Bangalore      1       CPU
1    Ankit  Bangalore      1       CPU
2    Ankit  Bangalore      2   DESKTOP
3   Bikaas  Bangalore      2   DESKTOP
4   Bikaas  Bangalore      2   DESKTOP
5   Bikaas  Bangalore      1   MONITOR


In [56]:
# my sol
query = '''
with cte as
(select 
emp_name, floor, count(*) as floor_freq,
row_number() over(partition by emp_name order by count(*) desc) as rn,
group_concat(distinct resources) as resources
from entries
group by emp_name, floor)

select 
emp_name, sum(floor_freq) as total_visits, floor as most_visited_floor,
group_concat(distinct resources) as resources_used
from cte
group by emp_name
'''
sqldf(query)

,emp_name,total_visits,most_visited_floor,resources_used
0,Ankit,3,1,"CPU,DESKTOP"
1,Bikaas,3,2,"DESKTOP,MONITOR"


In [57]:
# provided sol
query = '''
with cte_resources as (
select emp_name ,count(*) as total_visits ,group_concat(distinct resources) as resources_used
from entries
group by emp_name
),
floor_visits as (
select emp_name,floor,count(*) as no_of_floor_visits
,row_number() over(partition by emp_name order by count(*) desc) as rn
from entries
group by emp_name,floor
)
select fv.emp_name,r.total_visits,fv.floor as most_visited_floor,r.resources_used
 from floor_visits fv 
 inner join cte_resources r on fv.emp_name=r.emp_name
 where rn=1
'''
sqldf(query)

,emp_name,total_visits,most_visited_floor,resources_used
0,Ankit,3,1,"CPU,DESKTOP"
1,Bikaas,3,2,"DESKTOP,MONITOR"


#### [57 - Prime Subscription](https://100daysofsql.com/new-courses/1/coding-test/58)

In [ ]:
# Amazon, the world's largest online retailer, offers various services to its customers, including Amazon Prime membership, Video streaming, Amazon Music, Amazon Pay, and more. The company is interested in analyzing which of its services are most effective at converting regular customers into Amazon Prime members.

# You are given a table of events which consists services accessed by each users along with service access date. This table also contains the event when customer bought the prime membership.

# Table: Users

# user_id	name
# 1	Saurabh
# 2	Amit
# 3	Ankit
 

# Table : Events

# user_id	type	access_date
# 1	Amazon Music	2024-01-05
# 1	Amazon Video	2024-01-07
# 1	Prime	2024-01-08
# 1	Amazon Video	2024-01-09
# 2	Amazon Pay	2024-01-08
# 2	Prime	2024-01-09
# 3	Amazon Pay	2024-01-07
# 3	Amazon Music	2024-01-09
 

# Write an SQL to get date when each customer became prime member, last service used (just before becoming prime member), last service access date ( just before becoming prime member).

# If a customer never became prime member then populate last service used and last service access date by the customer.

 

# Expected Output:

# user_name	prime_member_date	last_access_service	last_access_service_date
# Saurabh	2024-01-08	Amazon Video	2024-01-07
# Amit	2024-01-09	Amazon Pay	2024-01-08
# Ankit	 	Amazon Music	2024-01-09

In [36]:
import pandas as pd

# Define the data for the Users table
users_data = {
    'user_id': [1, 2, 3],
    'name': ['Saurabh', 'Amit', 'Ankit']
}

# Create the DataFrame for Users
users = pd.DataFrame(users_data)

# Define the data for the Events table
events_data = {
    'user_id': [1, 1, 1, 1, 2, 2, 3, 3],
    'type': [
        'Amazon Music', 'Amazon Video', 'Prime', 'Amazon Video',
        'Amazon Pay', 'Prime', 'Amazon Pay', 'Amazon Music'
    ],
    'access_date': [
        '2024-01-05', '2024-01-07', '2024-01-08', '2024-01-09',
        '2024-01-08', '2024-01-09', '2024-01-07', '2024-01-09'
    ]
}

# Create the DataFrame for Events
events = pd.DataFrame(events_data)

# Convert 'access_date' to datetime
events['access_date'] = pd.to_datetime(events['access_date'])

# Display the DataFrames
print("Users DataFrame:")
display(users)
print("\nEvents DataFrame:")
display(events)


Users DataFrame:


,user_id,name
0,1,Saurabh
1,2,Amit
2,3,Ankit



Events DataFrame:


,user_id,type,access_date
0,1,Amazon Music,2024-01-05
1,1,Amazon Video,2024-01-07
2,1,Prime,2024-01-08
3,1,Amazon Video,2024-01-09
4,2,Amazon Pay,2024-01-08
5,2,Prime,2024-01-09
6,3,Amazon Pay,2024-01-07
7,3,Amazon Music,2024-01-09


In [37]:
# my sol
query = '''
with cte as 
(select *
,lag(type,1) over(partition by user_id order by access_date) as prev_type
,lag(access_date,1) over(partition by user_id order by access_date) as prev_access_date
,row_number() over(partition by user_id order by access_date desc) as rn
,max(case when type='Prime' then 'prime' else 'aprime' end ) over(partition by user_id) as prime_filter
from events)

-- select * from cte

select name,
max(case when type = 'Prime' then access_date end) as prime_member_date,
max(case when (type = 'Prime')  then prev_type when (rn=1 and type!='Prime' and prime_filter!='prime') then type else '' end) as last_access_service,
max(case when (type = 'Prime')  then prev_access_date when (rn=1 and type!='Prime' and prime_filter!='prime') then access_date else '' end) as last_access_service_date
-- max(case when (type = 'Prime')  then prev_access_date else '' end) as last_access_service_date
from cte join users u on cte.user_id=u.user_id
group by cte.user_id, name
'''
sqldf(query)

,name,prime_member_date,last_access_service,last_access_service_date
0,Saurabh,2024-01-08 00:00:00.000000,Amazon Video,2024-01-07 00:00:00.000000
1,Amit,2024-01-09 00:00:00.000000,Amazon Pay,2024-01-08 00:00:00.000000
2,Ankit,None,Amazon Music,2024-01-09 00:00:00.000000


In [38]:
# provided sol
query = '''
with cte as (
select *
, lag(type,1) over(partition by user_id order by access_date) as prev_type
, lag(access_date,1) over(partition by user_id order by access_date) as prev_access_date
,row_number() over(partition by user_id order by access_date desc) as rn
from events
)
select u.name as user_name 
, cte.access_date as prime_member_date , coalesce(cte.prev_type,c.type) as last_access_service, coalesce(cte.prev_access_date,c.access_date) as last_access_service_date
from users u
left join cte on u.user_id=cte.user_id and cte.type='Prime'
left join cte c on c.user_id=u.user_id and c.rn=1;
'''
sqldf(query)

,user_name,prime_member_date,last_access_service,last_access_service_date
0,Saurabh,2024-01-08 00:00:00.000000,Amazon Video,2024-01-07 00:00:00.000000
1,Amit,2024-01-09 00:00:00.000000,Amazon Pay,2024-01-08 00:00:00.000000
2,Ankit,None,Amazon Music,2024-01-09 00:00:00.000000


#### [58 - Penultimate Order](https://100daysofsql.com/new-courses/1/coding-test/59)

In [ ]:
# You are a data analyst working for an e-commerce company, responsible for analyzing customer orders to gain insights into their purchasing behavior. Your task is to write a SQL query to retrieve the details of the penultimate order for each customer. However, if a customer has placed only one order, you need to retrieve the details of that order instead.

# Table: orders (primary key : order_id)

# column name	datatype
# order_id	int
# order_date	date
# customer_name	varchar(20)
# product_name	varchar(20)
# sales	int


# Expected Output:

# order_id	order_date	customer_name	product_name
# 2	2023-01-02	Alexa	boAt
# 6	2023-01-03	Neha	Dress
# 4	2023-01-01	Ramesh	Titan

In [42]:
import pandas as pd

# Define the data
data = {
    'order_id': [1, 2, 3, 4, 4, 6],
    'order_date': [
        '2023-01-01', '2023-01-02', '2023-01-03', 
        '2023-01-01', '2023-01-02', '2023-01-03'
    ],
    'customer_name': ['Alexa', 'Alexa', 'Alexa', 'Ramesh', 'Ramesh', 'Neha'],
    'product_name': ['iphone', 'boAt', 'Rolex', 'Titan', 'Shirt', 'Dress'],
    'sales': [100, 300, 400, 200, 300, 100]
}

# Create the DataFrame
orders = pd.DataFrame(data)

# Convert 'order_date' to datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Display the DataFrame
display(orders)


,order_id,order_date,customer_name,product_name,sales
0,1,2023-01-01,Alexa,iphone,100
1,2,2023-01-02,Alexa,boAt,300
2,3,2023-01-03,Alexa,Rolex,400
3,4,2023-01-01,Ramesh,Titan,200
4,4,2023-01-02,Ramesh,Shirt,300
5,6,2023-01-03,Neha,Dress,100


In [43]:
# my sol
query = '''
with cte as
(select *
, dense_rank() over(partition by customer_name order by order_date desc) as order_rank
, count(*) over(partition by customer_name) as total_orders
 from orders)

select order_id, order_date,
customer_name, product_name
from cte
where order_rank=2 or total_orders=1
'''
sqldf(query)

,order_id,order_date,customer_name,product_name
0,2,2023-01-02 00:00:00.000000,Alexa,boAt
1,6,2023-01-03 00:00:00.000000,Neha,Dress
2,4,2023-01-01 00:00:00.000000,Ramesh,Titan


In [ ]:
# provided sol
query = '''
with cte as (
select * 
,row_number() over(partition by customer_name order by order_date desc) as rn 
,count(*) over(partition by customer_name) as cnt_of_orders
from orders
)
select order_id,order_date,customer_name,product_name
from cte
where rn=2 or cnt_of_orders=1;
'''
sqldf(query)

In [ ]:
#### 

#### [59 - Service Downtime](https://100daysofsql.com/new-courses/1/coding-test/60)

In [ ]:
# You are a DevOps engineer responsible for monitoring the health and status of various services in your organization's infrastructure. Your team conducts canary tests on each service every minute to ensure their reliability and performance. As part of your responsibilities, you need to develop a SQL to identify any service that experiences continuous downtime for atleast 5 minutes so that team can find the root cause and fix the issue.

# Table:service_status 

# column name	datatype
# service_name 	varchar(20)
# updated_time 	datetime
# status 	varchar(5)
 

# Expected Output:

# service_name	down_start_time	down_end_time	down_minutes
# hdfs	2024-03-06 10:02:00	2024-03-06 10:07:00	6
# hive	2024-03-06 10:03:00	2024-03-06 10:07:00	5

In [44]:
import pandas as pd

# Define the data
data = {
    'service_name': [
        'hdfs', 'hdfs', 'hdfs', 'hdfs', 'hdfs', 'hdfs', 'hdfs', 'hdfs', 'hdfs',
        'hive', 'hive', 'hive', 'hive', 'hive', 'hive', 'hive', 'hive', 'hive'
    ],
    'updated_time': [
        '2024-03-06 10:01:00', '2024-03-06 10:02:00', '2024-03-06 10:03:00', '2024-03-06 10:04:00', 
        '2024-03-06 10:05:00', '2024-03-06 10:06:00', '2024-03-06 10:07:00', '2024-03-06 10:08:00', 
        '2024-03-06 10:09:00', '2024-03-06 10:01:00', '2024-03-06 10:02:00', '2024-03-06 10:03:00', 
        '2024-03-06 10:04:00', '2024-03-06 10:05:00', '2024-03-06 10:06:00', '2024-03-06 10:07:00', 
        '2024-03-06 10:08:00', '2024-03-06 10:09:00'
    ],
    'status': [
        'up', 'down', 'down', 'down', 'down', 'down', 'down', 'up', 'down',
        'down', 'up', 'down', 'down', 'down', 'down', 'down', 'up', 'down'
    ]
}

# Create the DataFrame
service_status = pd.DataFrame(data)

# Convert 'updated_time' to datetime
service_status['updated_time'] = pd.to_datetime(service_status['updated_time'])

# Display the DataFrame
print(service_status)


   service_name        updated_time status
0          hdfs 2024-03-06 10:01:00     up
1          hdfs 2024-03-06 10:02:00   down
2          hdfs 2024-03-06 10:03:00   down
3          hdfs 2024-03-06 10:04:00   down
4          hdfs 2024-03-06 10:05:00   down
5          hdfs 2024-03-06 10:06:00   down
6          hdfs 2024-03-06 10:07:00   down
7          hdfs 2024-03-06 10:08:00     up
8          hdfs 2024-03-06 10:09:00   down
9          hive 2024-03-06 10:01:00   down
10         hive 2024-03-06 10:02:00     up
11         hive 2024-03-06 10:03:00   down
12         hive 2024-03-06 10:04:00   down
13         hive 2024-03-06 10:05:00   down
14         hive 2024-03-06 10:06:00   down
15         hive 2024-03-06 10:07:00   down
16         hive 2024-03-06 10:08:00     up
17         hive 2024-03-06 10:09:00   down


In [45]:
# my sol
query = '''
with cte as 
(select *
, row_number() over(partition by service_name order by updated_time) as service_rn
, row_number() over(partition by service_name, status order by updated_time) as status_rn
from service_status
)

select 
service_name, 
min(updated_time) as down_start_time
,max(updated_time) as down_start_time
, count(*) as down_minutes

from cte
where status = 'down'
group by service_name, service_rn-status_rn
having count(*)>=5
'''
sqldf(query)

,service_name,down_start_time,down_start_time,down_minutes
0,hdfs,2024-03-06 10:02:00.000000,2024-03-06 10:07:00.000000,6
1,hive,2024-03-06 10:03:00.000000,2024-03-06 10:07:00.000000,5


In [46]:
# provided sol
query = '''
WITH consecutive_down AS (
    SELECT 
        service_name,
        updated_time,
        status,
        ROW_NUMBER() OVER (PARTITION BY service_name ORDER BY updated_time) AS row_num,
        ROW_NUMBER() OVER (PARTITION BY service_name, status ORDER BY updated_time) AS status_row_num
    FROM
        service_status
)
SELECT 
    service_name,
    MIN(updated_time) AS down_start_time,
    MAX(updated_time) AS down_end_time,
    COUNT(*) AS down_minutes
FROM 
    consecutive_down
WHERE 
    status = 'down'
GROUP BY 
    service_name, 
    status_row_num - row_num
HAVING 
    COUNT(*) >= 5;
'''
sqldf(query)

,service_name,down_start_time,down_end_time,down_minutes
0,hdfs,2024-03-06 10:02:00.000000,2024-03-06 10:07:00.000000,6
1,hive,2024-03-06 10:03:00.000000,2024-03-06 10:07:00.000000,5


#### [60 - Fake Ratings](https://100daysofsql.com/new-courses/1/coding-test/61)

In [ ]:
# As an analyst at Amazon, you are responsible for ensuring the integrity of product ratings on the platform. Fake ratings can distort the perception of product quality and mislead customers. To maintain trust and reliability, you need to identify potential fake ratings that deviate significantly from the average ratings for each product.

# Write an SQL query to identify the single rating that is farthest (in absolute value) from the average rating value for each product.

# Table : product_ratings (primary key:rating_id)

# column name	datatype
# rating_id	int
# product_id 	int
# user_id 	int
# rating 	decimal(2,1)
 

# Expected Output:

# rating_id	product_id	user_id	rating
# 5	101	1005	3.2
# 6	102	1006	4.7

In [47]:
import pandas as pd

# Define the data
data = {
    'rating_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'product_id': [101, 101, 101, 101, 101, 102, 102, 102, 102, 102],
    'user_id': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    'rating': [4.5, 4.8, 4.9, 5.0, 3.2, 4.7, 4.0, 4.1, 3.8, 3.9]
}

# Create the DataFrame
product_ratings = pd.DataFrame(data)

# Display the DataFrame
print(product_ratings)


   rating_id  product_id  user_id  rating
0          1         101     1001     4.5
1          2         101     1002     4.8
2          3         101     1003     4.9
3          4         101     1004     5.0
4          5         101     1005     3.2
5          6         102     1006     4.7
6          7         102     1007     4.0
7          8         102     1008     4.1
8          9         102     1009     3.8
9         10         102     1010     3.9


In [53]:
# my sol
query = '''
with rating_diff_tbl as
(select *,
abs(rating-avg(rating) over(partition by product_id)) as rating_diff
from product_ratings)

,rating_diff_rnk as
(select rating_id, product_id, user_id, rating,
row_number() over(partition by product_id order by rating_diff desc) as rn
from rating_diff_tbl
)

select 
rating_id, product_id, user_id, rating
from rating_diff_rnk
where rn=1
'''
sqldf(query)

,rating_id,product_id,user_id,rating
0,5,101,1005,3.2
1,6,102,1006,4.7


In [52]:
# provided sol
query = '''
WITH rating_stats AS (
    SELECT product_id,AVG(rating) AS avg_rating
    FROM product_ratings
    GROUP BY product_id
),
farthest_rating AS (
    SELECT
        pr.rating_id,
        pr.product_id,
        pr.user_id,
        pr.rating,
        rs.avg_rating,
        ABS(pr.rating - rs.avg_rating) AS rating_difference,
        ROW_NUMBER() OVER (PARTITION BY pr.product_id ORDER BY ABS(pr.rating - rs.avg_rating) DESC) AS rn
    FROM
        product_ratings pr
    JOIN
        rating_stats rs ON pr.product_id = rs.product_id
)
SELECT
    rating_id,
    product_id,
    user_id,
    rating
FROM
    farthest_rating
WHERE
    rn = 1;
'''
sqldf(query)


,rating_id,product_id,user_id,rating
0,5,101,1005,3.2
1,6,102,1006,4.7


#### [61 - Food Preparation Time](https://100daysofsql.com/new-courses/1/coding-test/62)

In [ ]:
# You're analyzing the efficiency of food delivery on Zomato, focusing on the time taken by restaurants to prepare orders. Total food delivery time for an order is a combination of food preparation time + time taken by rider to deliver the order. 

# Write an SQL to calculate average food preparation time(in minutes) for each restaurant . Round the average to 2 decimal points and sort the output in increasing order of avg time. 

# Table: orders (primary key : order_id)

# column name	datatype
# order_id	int
# restaurant_id 	int
# order_time	time
# expected_delivery_time 	time
# actual_delivery_time 	time
# rider_delivery_mins 	int
 

# Expected Output:

# restaurant_id	avg_food_prep_mins
# 103	27.67
# 102	28.67
# 101	29.5

In [3]:
import pandas as pd

# Define the data
data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'restaurant_id': [101, 102, 103, 101, 102, 103, 101, 102, 103, 101],
    'order_time': ['12:00:00', '12:15:00', '12:30:00', '12:45:00', '13:00:00', '13:15:00', '13:30:00', '13:45:00', '14:00:00', '14:15:00'],
    'expected_delivery_time': ['12:30:00', '12:45:00', '13:00:00', '13:15:00', '13:30:00', '13:45:00', '14:00:00', '14:15:00', '14:30:00', '14:45:00'],
    'actual_delivery_time': ['12:45:00', '12:55:00', '13:10:00', '13:21:00', '13:36:00', '13:58:00', '14:12:00', '14:25:00', '14:30:00', '15:05:00'],
    'rider_delivery_mins': [15, 10, 15, 5, 10, 10, 20, 10, 5, 15]
}

# Create the DataFrame
orders = pd.DataFrame(data)

# Display the DataFrame
display(orders)


,order_id,restaurant_id,order_time,expected_delivery_time,actual_delivery_time,rider_delivery_mins
0,1,101,12:00:00,12:30:00,12:45:00,15
1,2,102,12:15:00,12:45:00,12:55:00,10
2,3,103,12:30:00,13:00:00,13:10:00,15
3,4,101,12:45:00,13:15:00,13:21:00,5
4,5,102,13:00:00,13:30:00,13:36:00,10
5,6,103,13:15:00,13:45:00,13:58:00,10
6,7,101,13:30:00,14:00:00,14:12:00,20
7,8,102,13:45:00,14:15:00,14:25:00,10
8,9,103,14:00:00,14:30:00,14:30:00,5
9,10,101,14:15:00,14:45:00,15:05:00,15


In [4]:
query = '''
select restaurant_id,
round(avg((julianday(actual_delivery_time)-julianday(order_time))*24*60-rider_delivery_mins),2) as avg_time
from orders
group by restaurant_id
order by avg_time asc
'''
sqldf(query)

,restaurant_id,avg_time
0,103,27.67
1,102,28.67
2,101,29.50


#### [62 - Late Food Deliveries](https://100daysofsql.com/new-courses/1/coding-test/63)

In [ ]:
# You're analyzing the efficiency of food delivery on Zomato, focusing on the late deliveries. Total food delivery time for an order is a combination of food preparation time + time taken by rider to deliver the order. 

# Suppose that as per expected time of delivery there is equal time allocated to restaurant for food preparation and rider to deliver the order.  Write an SQL to find orders which got delayed just because of more than expected time taken by the rider. 

# Table: orders (primary key : order_id)

# column name	datatype
# order_id	int
# restaurant_id 	int
# order_time	time
# expected_delivery_time 	time
# actual_delivery_time 	time
# rider_delivery_mins 	int
 

# Expected Output:

# order_id	expected_delivery_mins	rider_delivery_mins	food_prep_mins
# 2	25.0	30	10.0
# 6	30.0	29	14.0

In [6]:
import pandas as pd

# Define the data
data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'restaurant_id': [101, 102, 103, 101, 102, 103, 101, 102, 103, 101],
    'order_time': ['12:00:00', '12:15:00', '12:30:00', '12:45:00', '13:00:00', '13:15:00', '13:30:00', '13:45:00', '14:00:00', '14:15:00'],
    'expected_delivery_time': ['12:30:00', '12:40:00', '13:05:00', '13:12:00', '13:30:00', '13:45:00', '14:00:00', '14:20:00', '14:32:00', '14:50:00'],
    'actual_delivery_time': ['12:25:00', '12:55:00', '13:10:00', '13:21:00', '13:36:00', '13:58:00', '14:12:00', '14:25:00', '14:32:00', '15:05:00'],
    'rider_delivery_mins': [18, 30, 15, 5, 10, 29, 20, 10, 5, 18]
}

# Create the DataFrame
orders = pd.DataFrame(data)

# Display the DataFrame
display(orders)


,order_id,restaurant_id,order_time,expected_delivery_time,actual_delivery_time,rider_delivery_mins
0,1,101,12:00:00,12:30:00,12:25:00,18
1,2,102,12:15:00,12:40:00,12:55:00,30
2,3,103,12:30:00,13:05:00,13:10:00,15
3,4,101,12:45:00,13:12:00,13:21:00,5
4,5,102,13:00:00,13:30:00,13:36:00,10
5,6,103,13:15:00,13:45:00,13:58:00,29
6,7,101,13:30:00,14:00:00,14:12:00,20
7,8,102,13:45:00,14:20:00,14:25:00,10
8,9,103,14:00:00,14:32:00,14:32:00,5
9,10,101,14:15:00,14:50:00,15:05:00,18


In [7]:
query = '''

select 
order_id, round((julianday(expected_delivery_time)-julianday(order_time))*24*60,0) as expected_delivery_mins,
rider_delivery_mins,
round((julianday(actual_delivery_time)-julianday(order_time))*24*60-rider_delivery_mins,0) as food_prep_mins
from orders
where (julianday(expected_delivery_time)-julianday(order_time))*24*60<(julianday(actual_delivery_time)-julianday(order_time))*24*60
and rider_delivery_mins>((julianday(actual_delivery_time)-julianday(order_time))*24*60)/2

'''
sqldf(query)

,order_id,expected_delivery_mins,rider_delivery_mins,food_prep_mins
0,2,25.0,30,10.0
1,6,30.0,29,14.0


#### [63 - Country Indicators](https://100daysofsql.com/new-courses/1/coding-test/64)

In [ ]:
# In the realm of global indicators and country-level assessments, it's imperative to identify the years in which certain indicators hit their lowest values for each country. Leveraging a dataset provided by government, which contains indicators across multiple years for various countries, your task is to formulate an SQL query to find the following information:

# For each country and indicator combination, determine the year in which the indicator value was lowest, along with the corresponding indicator value.

# Sort the output by country name and indicator name.

# Table: country_data 

# column name	datatype
# country_name 	varchar(15)
# indicator_name 	varchar(30)
# year_2010 	decimal(3,2)
# year_2011 	decimal(3,2)
# year_2012 	decimal(3,2)
# year_2013 	decimal(3,2)
# year_2014 	decimal(3,2)
 

# Expected Output:

# country_name	indicator_name	indicator_value	year_number
# Canada	Control of Corruption	1.46	2010
# Canada	Government Effectiveness	1.35	2013
# Canada	Regulatory Quality	1.38	2010
# Canada	Rule of Law	1.42	2010
# Canada	Voice and Accountability	1.09	2014
# United States	Control of Corruption	1.26	2010
# United States	Government Effectiveness	1.25	2013
# United States	Regulatory Quality	1.28	2010
# United States	Rule of Law	1.32	2010
# United States	Voice and Accountability	1.05	2014

In [10]:
import pandas as pd

# Define the data
data = {
    'country_name': ['United States', 'United States', 'United States', 'United States', 'United States', 
                     'Canada', 'Canada', 'Canada', 'Canada', 'Canada'],
    'indicator_name': ['Control of Corruption', 'Government Effectiveness', 'Regulatory Quality', 'Rule of Law', 'Voice and Accountability',
                       'Control of Corruption', 'Government Effectiveness', 'Regulatory Quality', 'Rule of Law', 'Voice and Accountability'],
    'year_2010': [1.26, 1.27, 1.28, 1.32, 1.3, 1.46, 1.47, 1.38, 1.42, 1.4],
    'year_2011': [1.51, 1.45, 1.63, 1.61, 1.11, 1.61, 1.55, 1.73, 1.71, 1.19],
    'year_2012': [1.52, 1.28, 1.63, 1.6, 1.13, 1.71, 1.38, 1.63, 1.8, 1.21],
    'year_2013': [1.5, 1.25, 1.54, 1.54, 1.08, 1.5, 1.35, 1.59, 1.64, 1.16],
    'year_2014': [1.46, 1.27, 1.62, 1.62, 1.05, 1.56, 1.47, 1.68, 1.72, 1.09]
}

# Create the DataFrame
country_data = pd.DataFrame(data)

# Display the DataFrame
display(country_data)


,country_name,indicator_name,year_2010,year_2011,year_2012,year_2013,year_2014
0,United States,Control of Corruption,1.26,1.51,1.52,1.50,1.46
1,United States,Government Effectiveness,1.27,1.45,1.28,1.25,1.27
2,United States,Regulatory Quality,1.28,1.63,1.63,1.54,1.62
3,United States,Rule of Law,1.32,1.61,1.60,1.54,1.62
4,United States,Voice and Accountability,1.30,1.11,1.13,1.08,1.05
5,Canada,Control of Corruption,1.46,1.61,1.71,1.50,1.56
6,Canada,Government Effectiveness,1.47,1.55,1.38,1.35,1.47
7,Canada,Regulatory Quality,1.38,1.73,1.63,1.59,1.68
8,Canada,Rule of Law,1.42,1.71,1.80,1.64,1.72
9,Canada,Voice and Accountability,1.40,1.19,1.21,1.16,1.09


In [9]:
query = '''
with cte as 
(
    select country_name, indicator_name,year_2010 as indicator_value, 2010 as indicator_yr from country_data 
    union all
    select country_name, indicator_name,year_2011 as indicator_value, 2011 as indicator_yr from country_data 
    union all
    select country_name, indicator_name,year_2012 as indicator_value, 2012 as indicator_yr from country_data 
    union all
    select country_name, indicator_name,year_2013 as indicator_value, 2013 as indicator_yr from country_data 
    union all
    select country_name, indicator_name,year_2014 as indicator_value, 2014 as indicator_yr from country_data 
)

, cte2 as (select *,
dense_rank() over(partition by country_name, indicator_name order by indicator_value asc) as dr
from cte)

select 
country_name,	indicator_name,	indicator_value,	indicator_yr
from cte2
where dr=1
'''
sqldf(query)

,country_name,indicator_name,indicator_value,indicator_yr
0,Canada,Control of Corruption,1.46,2010
1,Canada,Government Effectiveness,1.35,2013
2,Canada,Regulatory Quality,1.38,2010
3,Canada,Rule of Law,1.42,2010
4,Canada,Voice and Accountability,1.09,2014
5,United States,Control of Corruption,1.26,2010
6,United States,Government Effectiveness,1.25,2013
7,United States,Regulatory Quality,1.28,2010
8,United States,Rule of Law,1.32,2010
9,United States,Voice and Accountability,1.05,2014


#### [64 - Loyal Customers](https://100daysofsql.com/new-courses/1/coding-test/65)

In [ ]:
In the domain of retail analytics, understanding customer behavior is crucial for driving business growth and enhancing customer satisfaction. One essential aspect is identifying customers who have purchased all available products, as they exhibit strong engagement with the product offerings and represent potential loyal customers.

 

Write an SQL to identify customers who have bought all the products available in products table along with total revenue generated by them. Sort the output by total_revenue in decreasing order.

Tables:

products (product_id, product_name, unit_price)

customers (customer_id, customer_name)

orders (order_id, customer_id, product_id, quantity, order_date)

 

Expected Output:

customer_name	total_revenue
Alice Johnson	1250
Somnath	1150

In [12]:
import pandas as pd

# Define the data for products table
products_data = {
    'product_id': [101, 102, 103],
    'product_name': ['Product A', 'Product B', 'Product C'],
    'unit_price': [150, 200, 250]
}

# Create the DataFrame for products
products = pd.DataFrame(products_data)

# Define the data for customers table
customers_data = {
    'customer_id': [1, 2, 3, 4],
    'customer_name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Somnath']
}

# Create the DataFrame for customers
customers = pd.DataFrame(customers_data)

# Define the data for orders table
orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'customer_id': [1, 1, 2, 2, 3, 3, 3, 1, 3, 4, 4, 4],
    'product_id': [101, 102, 101, 102, 103, 101, 102, 102, 103, 102, 101, 103],
    'quantity': [1, 1, 3, 2, 1, 2, 1, 1, 2, 1, 3, 2],
    'order_date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-07', '2024-01-08', '2024-01-09']
}

# Create the DataFrame for orders
orders = pd.DataFrame(orders_data)

# Display the DataFrames
print("Products DataFrame:")
display(products)
print("\nCustomers DataFrame:")
display(customers)
print("\nOrders DataFrame:")
display(orders)


Products DataFrame:


,product_id,product_name,unit_price
0,101,Product A,150
1,102,Product B,200
2,103,Product C,250



Customers DataFrame:


,customer_id,customer_name
0,1,John Doe
1,2,Jane Smith
2,3,Alice Johnson
3,4,Somnath



Orders DataFrame:


,order_id,customer_id,product_id,quantity,order_date
0,1,1,101,1,2024-01-01
1,2,1,102,1,2024-01-02
2,3,2,101,3,2024-01-03
3,4,2,102,2,2024-01-04
4,5,3,103,1,2024-01-05
5,6,3,101,2,2024-01-06
6,7,3,102,1,2024-01-07
7,8,1,102,1,2024-01-08
8,9,3,103,2,2024-01-09
9,10,4,102,1,2024-01-07


In [13]:
query = '''
with cte as 
(select 
customer_name ,o.customer_id, o.product_id, sum(quantity)*unit_price as revenue
from orders o join products p on o.product_id=p.product_id
join customers c on c.customer_id=o.customer_id
group by o.customer_id, o.product_id)

select customer_name, sum(revenue) as total_revenue
from cte 
group by customer_name
having count(distinct product_id)>=(select count(distinct product_id) from products)
'''
sqldf(query)

,customer_name,total_revenue
0,Alice Johnson,1250
1,Somnath,1150
